In [2]:
# Import all the necessary modules
import os
import sys
import os, sys
# from .../research/notebooks -> go up two levels to repo root
repo_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
import pandas_datareader as pdr
import math
import datetime
from datetime import datetime, timezone
import itertools
import ast
import yfinance as yf
import seaborn as sn
from IPython.display import display, HTML
from strategy_signal.trend_following_signal import (
    apply_jupyter_fullscreen_css, get_trend_donchian_signal_for_portfolio_with_rolling_r_sqr_vol_of_vol
)
from portfolio.strategy_performance import (calculate_sharpe_ratio, calculate_calmar_ratio, calculate_CAGR, calculate_risk_and_performance_metrics,
                                          calculate_compounded_cumulative_returns, estimate_fee_per_trade, rolling_sharpe_ratio)
from utils import coinbase_utils as cn
from portfolio import strategy_performance as perf
from sizing import position_sizing_binary_utils as size_bin
from sizing import position_sizing_continuous_utils as size_cont
from strategy_signal import trend_following_signal as tf
%matplotlib inline

In [3]:
import importlib
importlib.reload(cn)
importlib.reload(perf)
importlib.reload(tf)
importlib.reload(size_bin)
importlib.reload(size_cont)

<module 'sizing.position_sizing_continuous_utils' from '/Users/adheerchauhan/Documents/git/trend_following/sizing/position_sizing_continuous_utils.py'>

In [4]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
apply_jupyter_fullscreen_css()

## Helper Functions

In [6]:
from collections import OrderedDict

def print_strategy_params():
    """
    Pretty-print the strategy’s configuration values, with a blank line
    separating each logical section.
    """

    # ---- Define sections (title is just for dev readability) --------------
    sections = [
        ("Dates & universe", OrderedDict([
            ("start_date",      start_date),
            ("end_date",        end_date),
            ("warm_up_days",    WARMUP_DAYS),
            ("ticker_list",     ticker_list),
        ])),

        ("Moving-average / trend", OrderedDict([
            ("fast_mavg",                  fast_mavg),
            ("slow_mavg",                  slow_mavg),
            ("mavg_stepsize",              mavg_stepsize),
            ("mavg_z_score_window",        mavg_z_score_window),
            ("moving_avg_type",            moving_avg_type),
            ("ma_crossover_signal_weight", ma_crossover_signal_weight),
        ])),

        ("Donchian channel", OrderedDict([
            ("entry_rolling_donchian_window", entry_rolling_donchian_window),
            ("exit_rolling_donchian_window", exit_rolling_donchian_window),
            ("use_donchian_exit_gate", use_donchian_exit_gate),
            ("donchian_signal_weight",  donchian_signal_weight),
        ])),

        ("Volatility & risk", OrderedDict([
            ("volatility_window",            volatility_window),
            ("annualized_target_volatility", annualized_target_volatility),
            ("rolling_cov_window",           rolling_cov_window),
            ("rolling_atr_window",           rolling_atr_window),
            ("atr_multiplier",               atr_multiplier),
            ("log_std_window",               log_std_window),
            ("coef_of_variation_window",     coef_of_variation_window),
            ("vol_of_vol_z_score_window",    vol_of_vol_z_score_window),
            ("vol_of_vol_p_min",             vol_of_vol_p_min),
            ("r2_strong_threshold",          r2_strong_threshold)
        ])),

        ("Signal gating / quality", OrderedDict([
            ("lower_r_sqr_limit",             lower_r_sqr_limit),
            ("upper_r_sqr_limit",             upper_r_sqr_limit),
            ("rolling_r2_window",             rolling_r2_window),
            ("r2_smooth_window",              r2_smooth_window),
            ("r2_confirm_days",               r2_confirm_days),
            ("rolling_sharpe_window",         rolling_sharpe_window),
            ("use_activation",                use_activation),
            ("tanh_activation_constant_dict", tanh_activation_constant_dict),
            ("weighted_signal_ewm_window",    weighted_signal_ewm_window)
        ])),

        ("Trading toggles & thresholds", OrderedDict([
            ("long_only",                  long_only),
            ("use_coinbase_data",          use_coinbase_data),
            ("use_saved_files",            use_saved_files),
            ("saved_file_end_date",        saved_file_end_date),
            ("use_specific_start_date",    use_specific_start_date),
            ("signal_start_date",          signal_start_date),
            ("price_or_returns_calc",      price_or_returns_calc),
            ("notional_threshold_pct",     notional_threshold_pct),
            ("cooldown_counter_threshold", cooldown_counter_threshold),
            ("warmup_days",                WARMUP_DAYS)
        ])),

        ("Capital & execution", OrderedDict([
            ("initial_capital",        initial_capital),
            ("cash_buffer_percentage", cash_buffer_percentage),
            ("transaction_cost_est",   transaction_cost_est),
            ("passive_trade_rate",     passive_trade_rate),
            ("annual_trading_days",    annual_trading_days),
        ])),
    ]

    # ---- Compute width for neat alignment ---------------------------------
    longest_key = max(len(k) for _, sec in sections for k in sec)

    print("\nStrategy Parameters\n" + "-" * (longest_key + 30))
    for _, sec in sections:
        for k, v in sec.items():
            print(f"{k:<{longest_key}} : {v}")
        print()  # blank line between sections
    print("-" * (longest_key + 30) + "\n")

# ---------------------------------------------------------------------------
# Example usage (uncomment after your own parameter definitions are in scope)
# ---------------------------------------------------------------------------
# if __name__ == "__main__":
#     print_strategy_params()

In [7]:
def plot_signal_performance(df_1, df_2, ticker):

    fig = plt.figure(figsize=(20,12))
    layout = (2,2)
    signal_ax = plt.subplot2grid(layout, (0,0))
    price_ax = signal_ax.twinx()
    equity_curve_ax = plt.subplot2grid(layout, (0,1))
    sharpe_ax = plt.subplot2grid(layout, (1,0))
    portfolio_value_ax = plt.subplot2grid(layout, (1,1))

    _ = signal_ax.plot(df_1.index, df_1[f'{ticker}_final_signal'], label='Orig Signal', alpha=0.9)
    _ = signal_ax.plot(df_2.index, df_2[f'{ticker}_final_signal'], label='New Signal', alpha=0.9)
    _ = price_ax.plot(df_1.index, df_2[f'{ticker}_open'], label='Price', alpha=0.7, linestyle='--', color='magenta')
    _ = signal_ax.set_title(f'Orignal Signal vs New Signal')
    _ = signal_ax.set_ylabel('Signal')
    _ = signal_ax.set_xlabel('Date')
    _ = signal_ax.legend(loc='upper left')
    _ = signal_ax.grid()

    _ = equity_curve_ax.plot(df_1.index, df_1[f'equity_curve'], label='Orig Signal', alpha=0.9)
    _ = equity_curve_ax.plot(df_2.index, df_2[f'equity_curve'], label='New Signal', alpha=0.9)
    _ = equity_curve_ax.set_title(f'Equity Curve')
    _ = equity_curve_ax.set_ylabel('Equity Curve')
    _ = equity_curve_ax.set_xlabel('Date')
    _ = equity_curve_ax.legend(loc='upper left')
    _ = equity_curve_ax.grid()

    _ = sharpe_ax.plot(df_1.index, df_1[f'portfolio_rolling_sharpe_50'], label='Orig Signal', alpha=0.9)
    _ = sharpe_ax.plot(df_2.index, df_2[f'portfolio_rolling_sharpe_50'], label='New Signal', alpha=0.9)
    _ = sharpe_ax.set_title(f'Rolling Sharpe')
    _ = sharpe_ax.set_ylabel(f'Rolling Sharpe')
    _ = sharpe_ax.set_xlabel('Date')
    _ = sharpe_ax.legend(loc='upper left')
    _ = sharpe_ax.grid()

    _ = portfolio_value_ax.plot(df_1.index, df_1[f'total_portfolio_value'], label='Orig Signal', alpha=0.9)
    _ = portfolio_value_ax.plot(df_2.index, df_2[f'total_portfolio_value'], label='New Signal', alpha=0.9)
    _ = portfolio_value_ax.set_title(f'Total Portfolio Value')
    _ = portfolio_value_ax.set_ylabel('Portfolio Value')
    _ = portfolio_value_ax.set_xlabel('Date')
    _ = portfolio_value_ax.legend(loc='upper left')
    _ = portfolio_value_ax.grid()

    plt.tight_layout()

    return

# Walk Forward Analysis

In [14]:
import itertools
import time
import numpy as np
import pandas as pd

# ---------- Small helpers ----------
def zscore_in_fold(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    m, sd = s.mean(), s.std(ddof=0)
    if np.isfinite(sd) and sd > 0:
        return (s - m) / sd
    return pd.Series(0.0, index=s.index)

def composite_is_score(df_block: pd.DataFrame) -> pd.Series:
    # expects columns: annualized_sharpe_ratio, max_drawdown, trade_count
    z_turn = zscore_in_fold(df_block["trade_count"].fillna(df_block["trade_count"].median()))
    z_mdd  = zscore_in_fold(df_block["max_drawdown"].abs().fillna(df_block["max_drawdown"].abs().median()))
    score  = pd.to_numeric(df_block["annualized_sharpe_ratio"], errors="coerce") \
             - 0.25*z_turn - 0.15*z_mdd
    return score

def make_grid(fixed_params: dict, sweep_params: dict):
    """Yield parameter dicts: fixed merged with each sweep point."""
    keys, values = zip(*sweep_params.items()) if sweep_params else ([], [])
    for prod in (itertools.product(*values) if values else [()]):
        update = dict(zip(keys, prod))
        cfg = fixed_params.copy()
        cfg.update(update)
        yield cfg

def _grid_size(sweep_params: dict) -> int:
    if not sweep_params:
        return 1
    n = 1
    for v in sweep_params.values():
        n *= len(v)
    return n

def _fmt_params(p: dict) -> str:
    keys = [
        'annualized_target_volatility',
        'stop_loss_strategy',
        'atr_multiplier',
        'rolling_atr_window',
        'highest_high_window',
        'cooldown_counter_threshold',
    ]
    return ", ".join(f"{k}={p[k]}" for k in keys if k in p)

def _clean_params(p: dict) -> dict:
    """Remove fold-controlled keys so we can set them explicitly per fold."""
    drop = {"signal_start_date", "use_specific_start_date"}
    return {k: v for k, v in p.items() if k not in drop}

# ---------- Core WFA runner ----------
def run_wfa(
    start_date: str,
    end_date: str,
    ticker_list,
    *,
    is_months=18,
    os_months=3,
    step_equals_os=True,     # non-overlapping for speed
    warmup_days=300,
    min_trades_is=6,         # guardrail
    fixed_params: dict,
    sweep_params: dict,
    tf_fn,                   # your strategy function
    perf_fn,                 # your risk/perf metrics function (portfolio-level)
    include_ticker_metrics=True
):
    """
    Returns df_performance with IS and OS rows.
    Adds: is_score on IS rows; os_score (Sharpe) on OS rows; is_promoted flag on OS row.
    Uses YAML parameter names throughout.
    """
    start_ts = time.perf_counter()
    start_date = pd.Timestamp(start_date).date()
    end_date   = pd.Timestamp(end_date).date()

    IS_LEN = pd.DateOffset(months=is_months)
    OS_LEN = pd.DateOffset(months=os_months)
    STEP   = (OS_LEN if step_equals_os else pd.DateOffset(months=1))

    grid_n = _grid_size(sweep_params)
    print(f"[WFA] init: IS={is_months}m, OS={os_months}m, STEP={'OS_LEN' if step_equals_os else '1m'}, "
          f"warmup={warmup_days}d, tickers={len(ticker_list)}, grid_size={grid_n}", flush=True)

    perf_cols = [
        'sampling_category','start_date','end_date',
        'annualized_return','annualized_sharpe_ratio','calmar_ratio',
        'annualized_std_dev','max_drawdown','max_drawdown_duration',
        'hit_rate','t_statistic','p_value','trade_count',
        # params (YAML names)
        'annualized_target_volatility','stop_loss_strategy','atr_multiplier','rolling_atr_window',
        'highest_high_window','cooldown_counter_threshold',
        # scores
        'is_score','os_score','is_promoted'
    ]
    if include_ticker_metrics:
        ticker_perf_cols = ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']
        perf_cols += [f'{t}_{c}' for t in ticker_list for c in ticker_perf_cols]

    df_performance = pd.DataFrame(columns=perf_cols)

    fold_idx = 0
    start_is = start_date
    while True:
        end_is = (start_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_os = (end_is + pd.Timedelta(days=1))
        end_os = (start_os + OS_LEN - pd.Timedelta(days=1)).date()

        if end_os > end_date - pd.Timedelta(days=1):
            print("[WFA] done: OS end exceeds end_date; exiting loop.", flush=True)
            break

        fold_idx += 1
        print(f"\n[WFA] Fold {fold_idx}: IS {start_is} → {end_is} | OS {start_os} → {end_os} | warmup={warmup_days}d", flush=True)

        # ---- 1) Evaluate all sweep params in-sample ----
        is_rows = []
        print(f"[WFA] Fold {fold_idx}: evaluating {grid_n} config(s) in-sample…", flush=True)
        t0 = time.perf_counter()
        for j, p in enumerate(make_grid(fixed_params, sweep_params), start=1):
            p_is = _clean_params(p)
            print(f"[WFA] Fold {fold_idx} | IS cfg {j}/{grid_n}: {_fmt_params(p_is)}", flush=True)
            # 1a) In-sample run (pass fold-local signal_start_date)
            df_is = tf_fn(
                start_date=start_is - pd.Timedelta(days=warmup_days),
                end_date=end_is, ticker_list=ticker_list,
                **p_is,
                use_specific_start_date=True,
                signal_start_date=start_is,
            )
            df_is = df_is[df_is.index >= pd.Timestamp(start_is).date()]

            df_is = perf.calculate_asset_level_returns(df_is, end_is, ticker_list)

            # 1b) Portfolio metrics
            is_port = perf_fn(
                df_is,
                strategy_daily_return_col='portfolio_daily_pct_returns',
                strategy_trade_count_col='count_of_positions',
                include_transaction_costs_and_fees=False,
                passive_trade_rate=0.05, annual_trading_days=365,
                transaction_cost_est=0.001
            )
            print(f"[WFA] Fold {fold_idx} | IS cfg {j}/{grid_n} METRICS: "
                  f"Sharpe={is_port.get('annualized_sharpe_ratio',np.nan):.3f}, "
                  f"MDD={is_port.get('max_drawdown',np.nan):.3%}, "
                  f"trades={int(is_port.get('trade_count',0))}", flush=True)

            row = {
                'sampling_category':'in_sample',
                'start_date': start_is,
                'end_date': end_is,
                'is_score': np.nan, 'os_score': np.nan, 'is_promoted': False,
                **{k: p_is.get(k, np.nan) for k in [
                    'annualized_target_volatility','stop_loss_strategy','atr_multiplier',
                    'rolling_atr_window','highest_high_window','cooldown_counter_threshold'
                ]},
                **is_port
            }

            if include_ticker_metrics:
                for t in ticker_list:
                    tmet = perf.calculate_risk_and_performance_metrics(
                        df_is,
                        strategy_daily_return_col=f'{t}_daily_pct_returns',
                        strategy_trade_count_col=f'{t}_position_count',
                        annual_trading_days=365,
                        include_transaction_costs_and_fees=False
                    )
                    for c in ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']:
                        row[f'{t}_{c}'] = tmet[c]

            is_rows.append(row)

        print(f"[WFA] Fold {fold_idx}: IS grid complete in {time.perf_counter()-t0:.1f}s. Scoring…", flush=True)

        # attach IS rows and compute scores within the fold
        is_block = pd.DataFrame(is_rows)

        # guardrail: activity
        pre_n = len(is_block)
        is_block['ok'] = pd.to_numeric(is_block['trade_count'], errors='coerce').fillna(0) >= min_trades_is
        if is_block['ok'].any():
            is_block = is_block[is_block['ok']].copy()
            print(f"[WFA] Fold {fold_idx}: guardrail kept {len(is_block)}/{pre_n} configs (min_trades_is={min_trades_is}).", flush=True)
        else:
            print(f"[WFA] Fold {fold_idx}: guardrail filtered all {pre_n} configs; continuing with all (no guardrail).", flush=True)

        # IS composite score
        is_block['is_score'] = composite_is_score(is_block)

        # pick best by score (tie-breaks implicit by sort order)
        winner = is_block.sort_values(['is_score','annualized_sharpe_ratio','max_drawdown','trade_count'],
                                      ascending=[False, False, True, True]).iloc[0]

        print(f"[WFA] Fold {fold_idx}: WINNER (IS) "
              f"tv={winner.get('annualized_target_volatility')}, "
              f"stop={winner.get('stop_loss_strategy')}, "
              f"k={winner.get('atr_multiplier')}, "
              f"atr_w={winner.get('rolling_atr_window')}, "
              f"hh={winner.get('highest_high_window')}, "
              f"cd={winner.get('cooldown_counter_threshold')} | "
              f"score={winner['is_score']:.3f}, "
              f"Sharpe={winner['annualized_sharpe_ratio']:.3f}, "
              f"MDD={winner['max_drawdown']:.3%}, "
              f"trades={int(winner['trade_count'])}", flush=True)

        # append IS rows to df_performance
        df_performance = pd.concat([df_performance, is_block[perf_cols]], ignore_index=True)

        # ---- 2) Run the promoted config out-of-sample ----
        promoted_params = {
            'annualized_target_volatility': winner.get('annualized_target_volatility', fixed_params.get('annualized_target_volatility')),
            'stop_loss_strategy':           winner.get('stop_loss_strategy', fixed_params.get('stop_loss_strategy')),
            'rolling_atr_window':           winner.get('rolling_atr_window', fixed_params.get('rolling_atr_window')),
            'atr_multiplier':               winner.get('atr_multiplier', fixed_params.get('atr_multiplier')),
            'highest_high_window':          winner.get('highest_high_window', fixed_params.get('highest_high_window')),
            'cooldown_counter_threshold':   winner.get('cooldown_counter_threshold', fixed_params.get('cooldown_counter_threshold')),
        }
        p_os = fixed_params.copy()
        p_os.update(promoted_params)
        p_os = _clean_params(p_os)

        print(f"[WFA] Fold {fold_idx}: running OS with promoted params: {_fmt_params(p_os)}", flush=True)
        t1 = time.perf_counter()

        df_os = tf_fn(
            start_date=start_os - pd.Timedelta(days=warmup_days),
            end_date=end_os, ticker_list=ticker_list,
            **p_os,
            use_specific_start_date=True,
            signal_start_date=start_os,
        )
        df_os = df_os[df_os.index >= pd.Timestamp(start_os).date()]
        df_os = perf.calculate_asset_level_returns(df_os, end_os, ticker_list)

        os_port = perf_fn(
            df_os,
            strategy_daily_return_col='portfolio_daily_pct_returns',
            strategy_trade_count_col='count_of_positions',
            include_transaction_costs_and_fees=False,
            passive_trade_rate=0.05, annual_trading_days=365,
            transaction_cost_est=0.001
        )

        print(f"[WFA] Fold {fold_idx}: OS METRICS: Sharpe={os_port.get('annualized_sharpe_ratio',np.nan):.3f}, "
              f"MDD={os_port.get('max_drawdown',np.nan):.3%}, "
              f"trades={int(os_port.get('trade_count',0))} "
              f"(elapsed {time.perf_counter()-t1:.1f}s)", flush=True)

        os_row = {
            'sampling_category':'out_sample',
            'start_date': start_os,
            'end_date': end_os,
            'is_score': float(winner['is_score']),
            'os_score': float(os_port['annualized_sharpe_ratio']),  # OS score = OS Sharpe
            'is_promoted': True,
            **{k: p_os.get(k, np.nan) for k in [
                'annualized_target_volatility','stop_loss_strategy','atr_multiplier',
                'rolling_atr_window','highest_high_window','cooldown_counter_threshold'
            ]},
            **os_port
        }
        if include_ticker_metrics:
            for t in ticker_list:
                tmet = perf.calculate_risk_and_performance_metrics(
                    df_os,
                    strategy_daily_return_col=f'{t}_daily_pct_returns',
                    strategy_trade_count_col=f'{t}_position_count',
                    annual_trading_days=365,
                    include_transaction_costs_and_fees=False
                )
                for c in ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']:
                    os_row[f'{t}_{c}'] = tmet[c]

        df_performance.loc[len(df_performance)] = os_row

        print(f"[WFA] Fold {fold_idx}: complete. Rolling forward…", flush=True)
        # ---- roll forward ----
        start_is = (start_is + (OS_LEN if step_equals_os else pd.DateOffset(months=1))).date()

    print(f"\n[WFA] finished. folds={fold_idx}, total_rows={len(df_performance)} "
          f"(elapsed {time.perf_counter()-start_ts:.1f}s)", flush=True)

    return df_performance


## Target Volatility Walk Forward Analysis

In [76]:
fixed = dict(
    # your frozen prod config passed into tf_fn
    fast_mavg=20, slow_mavg=200, mavg_stepsize=8, mavg_z_score_window=126,
    entry_rolling_donchian_window=56, exit_rolling_donchian_window=28,
    use_donchian_exit_gate=False,
    ma_crossover_signal_weight=0.9, donchian_signal_weight=0.1, weighted_signal_ewm_window=4,
    rolling_r2_window=100, lower_r_sqr_limit=0.45, upper_r_sqr_limit=0.9,
    r2_smooth_window=3, r2_confirm_days=0, r2_strong_threshold=0.75,
    log_std_window=14, coef_of_variation_window=20, vol_of_vol_z_score_window=126, vol_of_vol_p_min=0.10,
    use_activation=False, tanh_activation_constant_dict=None,
    moving_avg_type='exponential', long_only=True, price_or_returns_calc='price',
    initial_capital=15000, rolling_cov_window=20, volatility_window=30,
    stop_loss_strategy='Chandelier', rolling_atr_window=14,
    atr_multiplier=2.25, highest_high_window=56, cooldown_counter_threshold=1,
    annualized_target_volatility=0.55,  # fallback if sweep doesn't set it
    transaction_cost_est=0.001, passive_trade_rate=0.05,
    notional_threshold_pct=0.10, rolling_sharpe_window=50, cash_buffer_percentage=0.10,
    annual_trading_days=365, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-07-31',
    # use_specific_start_date=True, signal_start_date='2022-04-01'
)
sweep = {"annualized_target_volatility":[0.40,0.50,0.55,0.60,0.70]}

df_tv = run_wfa(
    start_date="2022-04-01", end_date="2025-07-31",
    ticker_list=['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD'],
    is_months=18, os_months=3, step_equals_os=True, warmup_days=300,
    fixed_params=fixed, sweep_params=sweep,
    tf_fn=tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol,
    perf_fn=calculate_risk_and_performance_metrics,
    include_ticker_metrics=True
)


[WFA] init: IS=18m, OS=3m, STEP=OS_LEN, warmup=300d, tickers=5, grid_size=5

[WFA] Fold 1: IS 2022-04-01 → 2023-09-30 | OS 2023-10-01 → 2023-12-31 | warmup=300d
[WFA] Fold 1: evaluating 5 config(s) in-sample…
[WFA] Fold 1 | IS cfg 1/5: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=1
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
[WFA] Fold 1 | IS cfg 1/5 METRICS: Sharpe=0.150, MDD=-13.103%, trades=58
[WFA] Fold 1 | IS cfg 2/5: annualized_target_volatility=0.5, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=1
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getti

In [87]:
df_tv

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,in_sample,2022-04-01,2023-09-30,0.062030,0.150180,0.473397,0.386539,-0.131031,292 days,0.100365,0.556783,0.577903,58.0,0.40,Chandelier,2.25,14,56,1,0.423265,NaN,False,-0.007595,-0.439854,0.266515,-0.127637,0.020862,-0.594587,0.130958,-0.036686,0.039333,-0.111741,0.646857,-0.022147,-0.030570,-0.926539,0.293654,-0.111915,0.008067,-1.666147,0.091952,-0.014681,0.033652
1,in_sample,2022-04-01,2023-09-30,0.082740,0.251422,0.608804,0.485030,-0.135905,292 days,0.100365,0.604208,0.545956,57.0,0.50,Chandelier,2.25,14,56,1,0.574715,NaN,False,-0.001794,-0.368559,0.277505,-0.130493,0.027248,-0.362048,0.156629,-0.045688,0.043134,-0.028725,0.394093,-0.029176,-0.038148,-0.803790,0.364993,-0.138083,0.012181,-1.066856,0.124882,-0.025637,0.029939
2,in_sample,2022-04-01,2023-09-30,0.089059,0.273599,0.613952,0.583024,-0.145059,292 days,0.102190,0.604660,0.545656,57.0,0.55,Chandelier,2.25,14,56,1,0.480356,NaN,False,0.006367,-0.304262,0.274612,-0.122420,0.026697,-0.336930,0.170430,-0.048243,0.046199,0.010380,0.422925,-0.033944,-0.040931,-0.694602,0.427412,-0.156670,0.013448,-0.934778,0.137117,-0.028189,0.060043
3,in_sample,2022-04-01,2023-09-30,0.098075,0.307117,0.651815,0.623411,-0.150465,292 days,0.100365,0.635967,0.525064,59.0,0.60,Chandelier,2.25,14,56,1,0.220550,NaN,False,0.002035,-0.330487,0.281130,-0.124702,0.028099,-0.326153,0.165298,-0.039641,0.050368,0.053953,0.460905,-0.036930,-0.036057,-0.626262,0.442514,-0.161408,0.014388,-0.849444,0.146444,-0.030689,0.039018
4,in_sample,2022-04-01,2023-09-30,0.107947,0.337019,0.655483,0.660664,-0.164682,292 days,0.104015,0.652556,0.514317,63.0,0.70,Chandelier,2.25,14,56,1,-0.379549,NaN,False,0.013310,-0.235396,0.282342,-0.124732,0.026165,-0.360799,0.163055,-0.039586,0.059086,0.126599,0.523094,-0.042058,-0.051541,-0.651922,0.500961,-0.178276,0.022767,-0.598874,0.154056,-0.030714,0.056194
5,out_sample,2023-10-01,2023-12-31,8.559929,5.051270,123.856993,0.633967,-0.069111,22 days,0.500000,2.589370,0.011195,47.0,0.50,Chandelier,2.25,14,56,1,0.574715,5.051270,True,0.197930,0.805460,0.261972,-0.047620,-0.000114,-0.919299,0.093311,-0.022302,0.241325,1.392554,0.193601,-0.030912,1.585598,5.401248,0.192072,-0.027257,2.333130,4.255250,0.449820,-0.022730,0.267934
6,in_sample,2022-07-01,2023-12-31,0.515139,1.761367,5.204459,0.428444,-0.098980,153 days,0.178506,2.429623,0.015435,94.0,0.40,Chandelier,2.25,14,56,1,2.488981,NaN,False,0.019613,-0.155371,0.256914,-0.127798,0.019765,-0.581772,0.118209,-0.044809,0.070900,0.276397,0.345170,-0.044577,0.156967,1.083642,0.207667,-0.042807,0.194538,1.328109,0.305386,-0.016654,0.071111
7,in_sample,2022-07-01,2023-12-31,0.614936,1.684665,4.618796,0.560074,-0.133138,159 days,0.176685,2.281161,0.022922,108.0,0.50,Chandelier,2.25,14,56,1,1.686007,NaN,False,0.027858,-0.081462,0.272251,-0.130806,0.033380,-0.230939,0.138535,-0.045851,0.069797,0.228252,0.286334,-0.054929,0.185414,1.098212,0.259315,-0.057876,0.231545,1.331651,0.336869,-0.025111,0.120147
8,in_sample,2022-07-01,2023-12-31,0.671074,1.734672,4.973026,0.588984,-0.134943,159 days,0.178506,2.332251,0.020049,109.0,0.55,Chandelier,2.25,14,56,1,1

In [89]:
df_tv['vol_tracking_error'] = (np.abs(df_tv['annualized_std_dev'] - df_tv['annualized_target_volatility']) / df_tv['annualized_target_volatility'])

in_sample_cond = (df_tv.sampling_category == 'in_sample')
df_tv_is = df_tv[in_sample_cond].reset_index(drop=True)
df_tv_os = df_tv[~in_sample_cond].reset_index(drop=True)

In [91]:
df_tv.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Target_Volatility_Performance-2022-04-01-2025-10-01.pickle')

In [40]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'annualized_std_dev':['median','mean','std'],
            'vol_tracking_error':['median','mean','std'],
            'trade_count':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}

In [97]:
df_tv_is.groupby(['annualized_target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio                      \
                                              median      mean       std   
annualized_target_volatility                                               
0.40                                        1.761367  1.609636  0.686733   
0.55                                        1.734672  1.531630  0.587578   
0.50                                        1.684665  1.522892  0.601438   
0.60                                        1.674058  1.510240  0.570507   
0.70                                        1.631542  1.502700  0.559738   

                             annualized_return                      \
                                        median      mean       std   
annualized_target_volatility                                         
0.40                                  0.515139  0.513264  0.232470   
0.55                                  0.671074  0.618087  0.271165   
0.50                                  0.613093  0.580513  0.263852   
0.60                                  0.671616  0.628047  0.276296   
0.70                                  0.680846  0.656702  0.292981   

                             max_drawdown                      \
                                   median      mean       std   
annualized_target_volatility                                    
0.40                            -0.175480 -0.169407  0.049690   
0.55                            -0.227879 -0.212002  0.058041   
0.50                            -0.213488 -0.200800  0.055260   
0.60                            -0.237117 -0.221300  0.059913   
0.70                            -0.242743 -0.233133  0.059493   

                             annualized_std_dev                      \
                                         median      mean       std   
annualized_target_volatility                                          
0.40                                   0.428444  0.433328  0.029262   
0.55                                   0.583024  0.564907  0.032707   
0.50                                   0.493990  0.519426  0.037084   
0.60                                   0.612973  0.587195  0.042597   
0.70                                   0.636773  0.604931  0.066728   

                             vol_tracking_error                      \
                                         median      mean       std   
annualized_target_volatility                                          
0.40                                   0.071111  0.092936  0.058090   
0.55                                   0.060043  0.056659  0.025459   
0.50                                   0.029939  0.061661  0.053161   
0.60                                   0.048835  0.061054  0.034970   
0.70                                   0.090325  0.135813  0.095326   

                             trade_count                         \
                                  median        mean        std   
annualized_target_volatility                                      
0.40                               104.0   97.428571  20.525246   
0.55                               122.0  113.428571  27.748016   
0.50                               116.0  108.857143  25.281275   
0.60                               128.0  117.571429  28.970428   
0.70                               132.0  121.714286  28.848537   

                             BTC-USD_annualized_sharpe_ratio            \
                                                      median      mean   
annualized_target_volatility                                             
0.40                                               -0.155371  0.059218   
0.55                                               -0.039606  0.096297   
0.50                                               -0.081462  0.074563   
0.60                                               -0.035473  0.055127   
0.70                                                0.047208  0.097584   

                                       ETH-USD_annualized_sharpe_ratio

In [99]:
df_tv_os.groupby(['annualized_target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio                     \
                                              median      mean      std   
annualized_target_volatility                                              
0.5                                         5.051270  5.051270      NaN   
0.4                                        -0.836667 -1.175719  2.86716   

                             annualized_return                      \
                                        median      mean       std   
annualized_target_volatility                                         
0.5                                   8.559929  8.559929       NaN   
0.4                                  -0.079071  0.067714  0.518424   

                             max_drawdown                      \
                                   median      mean       std   
annualized_target_volatility                                    
0.5                             -0.069111 -0.069111       NaN   
0.4                             -0.095452 -0.091789  0.043373   

                             annualized_std_dev                      \
                                         median      mean       std   
annualized_target_volatility                                          
0.5                                    0.633967  0.633967       NaN   
0.4                                    0.300797  0.301143  0.117193   

                             vol_tracking_error                      \
                                         median      mean       std   
annualized_target_volatility                                          
0.5                                    0.267934  0.267934       NaN   
0.4                                    0.248007  0.291520  0.239067   

                             trade_count                        \
                                  median       mean        std   
annualized_target_volatility                                     
0.5                                 47.0  47.000000        NaN   
0.4                                 16.0  16.666667  10.033278   

                             BTC-USD_annualized_sharpe_ratio               \
                                                      median     mean std   
annualized_target_volatility                                                
0.5                                                  0.80546  0.80546 NaN   
0.4                                                 -1.81793     -inf NaN   

                             ETH-USD_annualized_sharpe_ratio                \
                                                      median      mean std   
annualized_target_volatility                                                 
0.5                                                -0.919299 -0.919299 NaN   
0.4                                                     -inf      -inf NaN   

                             SOL-USD_annualized_sharpe_ratio                \
                                                      median      mean std   
annualized_target_volatility                                                 
0.5                                                 1.392554  1.392554 NaN   
0.4                                                     -inf      -inf NaN   

                             ADA-USD_annualized_sharpe_ratio                \
                                                      median      mean std   
annualized_target_volatility                                                 
0.5                                                 5.401248  5.401248 NaN   
0.4                                                     -inf      -inf NaN   

                             AVAX-USD_annualized_sharpe_ratio               
                                                       median     mean std  
annualized_target_volatility                                                
0.5                                                   4.25525  4.25525 NaN  
0.4                                                      -inf     

In [101]:
df_tv_os

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,out_sample,2023-10-01,2023-12-31,8.559929,5.051270,123.856993,0.633967,-0.069111,22 days,0.500000,2.589370,0.011195,47.0,0.5,Chandelier,2.25,14,56,1,0.574715,5.051270,True,0.197930,0.805460,0.261972,-0.047620,-0.000114,-0.919299,0.093311,-0.022302,0.241325,1.392554,0.193601,-0.030912,1.585598,5.401248,0.192072,-0.027257,2.333130,4.255250,0.449820,-0.022730,0.267934
1,out_sample,2024-01-01,2024-03-31,1.008826,2.420412,6.985011,0.424984,-0.144427,37 days,0.373626,1.294133,0.198931,23.0,0.4,Chandelier,2.25,14,56,1,2.488981,2.420412,True,0.083092,0.336052,0.140872,-0.058698,0.936115,2.754706,0.299632,-0.109713,-0.026717,-7.974690,0.005347,-0.006729,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.062459
2,out_sample,2024-04-01,2024-06-30,-0.243350,-3.485933,-3.300537,0.136353,-0.073730,90 days,0.054945,-1.477854,0.142938,9.0,0.4,Chandelier,2.25,14,56,1,2.899567,-3.485933,True,-0.214787,-2.912832,0.246354,-0.058503,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.659119
3,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,2.25,14,56,1,2.781900,0.373578,True,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
4,out_sample,2024-10-01,2024-12-31,0.227801,0.821012,2.909473,0.337605,-0.078296,38 days,0.304348,0.523965,0.601577,32.0,0.4,Chandelier,2.25,14,56,1,2.466243,0.821012,True,0.470239,2.080154,0.240368,-0.034100,-0.047829,-3.438207,0.076056,-0.016832,0.054504,0.091746,0.188210,-0.042809,0.000000,-inf,NaN,0.000000,-0.041881,-1.600203,0.177761,-0.024942,0.155986
5,out_sample,2025-01-01,2025-03-31,-0.404289,-5.136470,-3.371731,0.215660,-0.119905,89 days,0.066667,-2.328531,0.022150,14.0,0.4,Chandelier,2.25,14,56,1,2.506616,-5.136470,True,-0.333097,-4.104783,0.225123,-0.095063,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,-0.022585,-6.327392,NaN,-0.005617,0.000000,-inf,NaN,0.000000,0.460850
6,out_sample,2025-04-01,2025-06-30,-0.267912,-2.046912,-2.379160,0.263989,-0.112608,47 days,0.263736,-0.878075,0.382241,18.0,0.4,Chandelier,2.25,14,56,1,2.175352,-2.046912,True,-0.035857,-0.723028,0.252532,-0.048488,0.028990,-0.274681,0.130054,-0.021096,0.051722,0.057077,0.114033,-0.014103,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.340028


In [103]:
df_tv_is

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,in_sample,2022-04-01,2023-09-30,0.062030,0.150180,0.473397,0.386539,-0.131031,292 days,0.100365,0.556783,0.577903,58.0,0.40,Chandelier,2.25,14,56,1,0.423265,NaN,False,-0.007595,-0.439854,0.266515,-0.127637,0.020862,-0.594587,0.130958,-0.036686,0.039333,-0.111741,0.646857,-0.022147,-0.030570,-0.926539,0.293654,-0.111915,0.008067,-1.666147,0.091952,-0.014681,0.033652
1,in_sample,2022-04-01,2023-09-30,0.082740,0.251422,0.608804,0.485030,-0.135905,292 days,0.100365,0.604208,0.545956,57.0,0.50,Chandelier,2.25,14,56,1,0.574715,NaN,False,-0.001794,-0.368559,0.277505,-0.130493,0.027248,-0.362048,0.156629,-0.045688,0.043134,-0.028725,0.394093,-0.029176,-0.038148,-0.803790,0.364993,-0.138083,0.012181,-1.066856,0.124882,-0.025637,0.029939
2,in_sample,2022-04-01,2023-09-30,0.089059,0.273599,0.613952,0.583024,-0.145059,292 days,0.102190,0.604660,0.545656,57.0,0.55,Chandelier,2.25,14,56,1,0.480356,NaN,False,0.006367,-0.304262,0.274612,-0.122420,0.026697,-0.336930,0.170430,-0.048243,0.046199,0.010380,0.422925,-0.033944,-0.040931,-0.694602,0.427412,-0.156670,0.013448,-0.934778,0.137117,-0.028189,0.060043
3,in_sample,2022-04-01,2023-09-30,0.098075,0.307117,0.651815,0.623411,-0.150465,292 days,0.100365,0.635967,0.525064,59.0,0.60,Chandelier,2.25,14,56,1,0.220550,NaN,False,0.002035,-0.330487,0.281130,-0.124702,0.028099,-0.326153,0.165298,-0.039641,0.050368,0.053953,0.460905,-0.036930,-0.036057,-0.626262,0.442514,-0.161408,0.014388,-0.849444,0.146444,-0.030689,0.039018
4,in_sample,2022-04-01,2023-09-30,0.107947,0.337019,0.655483,0.660664,-0.164682,292 days,0.104015,0.652556,0.514317,63.0,0.70,Chandelier,2.25,14,56,1,-0.379549,NaN,False,0.013310,-0.235396,0.282342,-0.124732,0.026165,-0.360799,0.163055,-0.039586,0.059086,0.126599,0.523094,-0.042058,-0.051541,-0.651922,0.500961,-0.178276,0.022767,-0.598874,0.154056,-0.030714,0.056194
5,in_sample,2022-07-01,2023-12-31,0.515139,1.761367,5.204459,0.428444,-0.098980,153 days,0.178506,2.429623,0.015435,94.0,0.40,Chandelier,2.25,14,56,1,2.488981,NaN,False,0.019613,-0.155371,0.256914,-0.127798,0.019765,-0.581772,0.118209,-0.044809,0.070900,0.276397,0.345170,-0.044577,0.156967,1.083642,0.207667,-0.042807,0.194538,1.328109,0.305386,-0.016654,0.071111
6,in_sample,2022-07-01,2023-12-31,0.614936,1.684665,4.618796,0.560074,-0.133138,159 days,0.176685,2.281161,0.022922,108.0,0.50,Chandelier,2.25,14,56,1,1.686007,NaN,False,0.027858,-0.081462,0.272251,-0.130806,0.033380,-0.230939,0.138535,-0.045851,0.069797,0.228252,0.286334,-0.054929,0.185414,1.098212,0.259315,-0.057876,0.231545,1.331651,0.336869,-0.025111,0.120147
7,in_sample,2022-07-01,2023-12-31,0.671074,1.734672,4.973026,0.588984,-0.134943,159 days,0.178506,2.332251,0.020049,109.0,0.55,Chandelier,2.25,14,56,1,1.688894,NaN,False,0.033853,-0.039606,0.273032,-0.125210,0.036170,-0.167746,0.149236,-0.048414,0.077300,0.279836,0.298430,-0.060002,0.205064,1.182378,0.272855,-0.055236,0.241804,1.344338,0.351631,-0.027465,0.070880
8,in_sample,2022-07-01,2023-12-31,0.711719,1.748907,4.943141,0.629301,-0.143981,159 days,0.174863,2.340624,0.019609,112.0,0.60,Chandelier,2.25,14,56,1,1.5346

## Cooldown Counter Threshold

In [56]:
import itertools
import time
import numpy as np
import pandas as pd

# ---------- Small helpers ----------
def zscore_in_fold(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    m, sd = s.mean(), s.std(ddof=0)
    if np.isfinite(sd) and sd > 0:
        return (s - m) / sd
    return pd.Series(0.0, index=s.index)

def composite_is_score(df_block: pd.DataFrame) -> pd.Series:
    # expects columns: annualized_sharpe_ratio, max_drawdown, trade_count
    z_turn = zscore_in_fold(df_block["trade_count"].fillna(df_block["trade_count"].median()))
    z_mdd  = zscore_in_fold(df_block["max_drawdown"].abs().fillna(df_block["max_drawdown"].abs().median()))
    score  = pd.to_numeric(df_block["annualized_sharpe_ratio"], errors="coerce") \
             - 0.25*z_turn - 0.15*z_mdd
    return score

def make_grid(fixed_params: dict, sweep_params: dict):
    """Yield parameter dicts: fixed merged with each sweep point."""
    keys, values = zip(*sweep_params.items()) if sweep_params else ([], [])
    for prod in (itertools.product(*values) if values else [()]):
        update = dict(zip(keys, prod))
        cfg = fixed_params.copy()
        cfg.update(update)
        yield cfg

def _grid_size(sweep_params: dict) -> int:
    if not sweep_params:
        return 1
    n = 1
    for v in sweep_params.values():
        n *= len(v)
    return n

def _fmt_params(p: dict) -> str:
    keys = [
        'annualized_target_volatility',
        'stop_loss_strategy',
        'atr_multiplier',
        'rolling_atr_window',
        'highest_high_window',
        'cooldown_counter_threshold',
    ]
    return ", ".join(f"{k}={p[k]}" for k in keys if k in p)

def _clean_params(p: dict) -> dict:
    """Remove fold-controlled keys so we can set them explicitly per fold."""
    drop = {"signal_start_date", "use_specific_start_date"}
    return {k: v for k, v in p.items() if k not in drop}

# ---------- Core WFA runner ----------
def run_wfa(
    start_date: str,
    end_date: str,
    ticker_list,
    *,
    is_months=18,
    os_months=3,
    step_equals_os=True,     # non-overlapping for speed
    warmup_days=300,
    min_trades_is=6,         # guardrail
    fixed_params: dict,
    sweep_params: dict,
    tf_fn,                   # your strategy function
    perf_fn,                 # your risk/perf metrics function (portfolio-level)
    include_ticker_metrics=True,
    promote_top_k: int = 1   # << NEW: promote top-k IS winners per fold into OS
):
    """
    Returns df_performance with IS and OS rows.
    Adds: is_score on IS rows; os_score (Sharpe) on OS rows; is_promoted flag on OS rows.
    Uses YAML parameter names throughout.
    NEW: promotes top-k IS configs per fold (promotion_rank, promotion_set_id).
    """
    start_ts = time.perf_counter()
    start_date = pd.Timestamp(start_date).date()
    end_date   = pd.Timestamp(end_date).date()

    IS_LEN = pd.DateOffset(months=is_months)
    OS_LEN = pd.DateOffset(months=os_months)
    STEP   = (OS_LEN if step_equals_os else pd.DateOffset(months=1))

    grid_n = _grid_size(sweep_params)
    print(f"[WFA] init: IS={is_months}m, OS={os_months}m, STEP={'OS_LEN' if step_equals_os else '1m'}, "
          f"warmup={warmup_days}d, tickers={len(ticker_list)}, grid_size={grid_n}, promote_top_k={promote_top_k}", flush=True)

    perf_cols = [
        'sampling_category','start_date','end_date',
        'annualized_return','annualized_sharpe_ratio','calmar_ratio',
        'annualized_std_dev','max_drawdown','max_drawdown_duration',
        'hit_rate','t_statistic','p_value','trade_count',
        # params (YAML names)
        'annualized_target_volatility','stop_loss_strategy','atr_multiplier','rolling_atr_window',
        'highest_high_window','cooldown_counter_threshold',
        # scores
        'is_score','os_score','is_promoted',
        # NEW
        'promotion_rank','promotion_set_id'
    ]
    if include_ticker_metrics:
        ticker_perf_cols = ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']
        perf_cols += [f'{t}_{c}' for t in ticker_list for c in ticker_perf_cols]

    df_performance = pd.DataFrame(columns=perf_cols)

    fold_idx = 0
    start_is = start_date
    while True:
        end_is = (start_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_os = (end_is + pd.Timedelta(days=1))
        end_os = (start_os + OS_LEN - pd.Timedelta(days=1)).date()

        if end_os > end_date - pd.Timedelta(days=1):
            print("[WFA] done: OS end exceeds end_date; exiting loop.", flush=True)
            break

        fold_idx += 1
        print(f"\n[WFA] Fold {fold_idx}: IS {start_is} → {end_is} | OS {start_os} → {end_os} | warmup={warmup_days}d", flush=True)

        # ---- 1) Evaluate all sweep params in-sample ----
        is_rows = []
        print(f"[WFA] Fold {fold_idx}: evaluating {grid_n} config(s) in-sample…", flush=True)
        t0 = time.perf_counter()
        for j, p in enumerate(make_grid(fixed_params, sweep_params), start=1):
            p_is = _clean_params(p)
            print(f"[WFA] Fold {fold_idx} | IS cfg {j}/{grid_n}: {_fmt_params(p_is)}", flush=True)
            # 1a) In-sample run (pass fold-local signal_start_date)
            df_is = tf_fn(
                start_date=start_is - pd.Timedelta(days=warmup_days),
                end_date=end_is, ticker_list=ticker_list,
                **p_is,
                use_specific_start_date=True,
                signal_start_date=start_is,
            )
            df_is = df_is[df_is.index >= pd.Timestamp(start_is).date()]

            df_is = perf.calculate_asset_level_returns(df_is, end_is, ticker_list)

            # 1b) Portfolio metrics
            is_port = perf_fn(
                df_is,
                strategy_daily_return_col='portfolio_daily_pct_returns',
                strategy_trade_count_col='count_of_positions',
                include_transaction_costs_and_fees=False,
                passive_trade_rate=0.05, annual_trading_days=365,
                transaction_cost_est=0.001
            )
            print(f"[WFA] Fold {fold_idx} | IS cfg {j}/{grid_n} METRICS: "
                  f"Sharpe={is_port.get('annualized_sharpe_ratio',np.nan):.3f}, "
                  f"MDD={is_port.get('max_drawdown',np.nan):.3%}, "
                  f"trades={int(is_port.get('trade_count',0))}", flush=True)

            row = {
                'sampling_category':'in_sample',
                'start_date': start_is,
                'end_date': end_is,
                'is_score': np.nan, 'os_score': np.nan, 'is_promoted': False,
                'promotion_rank': np.nan, 'promotion_set_id': None,
                **{k: p_is.get(k, np.nan) for k in [
                    'annualized_target_volatility','stop_loss_strategy','atr_multiplier',
                    'rolling_atr_window','highest_high_window','cooldown_counter_threshold'
                ]},
                **is_port
            }

            if include_ticker_metrics:
                for t in ticker_list:
                    tmet = perf.calculate_risk_and_performance_metrics(
                        df_is,
                        strategy_daily_return_col=f'{t}_daily_pct_returns',
                        strategy_trade_count_col=f'{t}_position_count',
                        annual_trading_days=365,
                        include_transaction_costs_and_fees=False
                    )
                    for c in ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']:
                        row[f'{t}_{c}'] = tmet[c]

            is_rows.append(row)

        print(f"[WFA] Fold {fold_idx}: IS grid complete in {time.perf_counter()-t0:.1f}s. Scoring…", flush=True)

        # attach IS rows and compute scores within the fold
        is_block = pd.DataFrame(is_rows)

        # guardrail: activity
        pre_n = len(is_block)
        is_block['ok'] = pd.to_numeric(is_block['trade_count'], errors='coerce').fillna(0) >= min_trades_is
        if is_block['ok'].any():
            is_block = is_block[is_block['ok']].copy()
            print(f"[WFA] Fold {fold_idx}: guardrail kept {len(is_block)}/{pre_n} configs (min_trades_is={min_trades_is}).", flush=True)
        else:
            print(f"[WFA] Fold {fold_idx}: guardrail filtered all {pre_n} configs; continuing with all (no guardrail).", flush=True)

        # IS composite score
        is_block['is_score'] = composite_is_score(is_block)

        # pick top-k by score (tie-breakers implicit)
        winners_block = is_block.sort_values(
            ['is_score','annualized_sharpe_ratio','max_drawdown','trade_count'],
            ascending=[False, False, True, True]
        ).head(max(1, int(promote_top_k))).copy()

        # annotate a promotion set id for this fold
        promotion_set_id = f"fold{fold_idx}_{start_os}"
        winners_block['promotion_rank'] = range(1, len(winners_block) + 1)
        winners_block['promotion_set_id'] = promotion_set_id

        # append IS rows to df_performance (for traceability)
        df_performance = pd.concat([df_performance, is_block[perf_cols]], ignore_index=True)

        print(f"[WFA] Fold {fold_idx}: promoting top-{len(winners_block)} to OS…", flush=True)
        for _, winner in winners_block.iterrows():
            print(f"[WFA] Fold {fold_idx}: WINNER rank {int(winner['promotion_rank'])} "
                  f"tv={winner.get('annualized_target_volatility')}, "
                  f"stop={winner.get('stop_loss_strategy')}, "
                  f"k={winner.get('atr_multiplier')}, "
                  f"atr_w={winner.get('rolling_atr_window')}, "
                  f"hh={winner.get('highest_high_window')}, "
                  f"cd={winner.get('cooldown_counter_threshold')} | "
                  f"IS score={winner['is_score']:.3f}, "
                  f"IS Sharpe={winner['annualized_sharpe_ratio']:.3f}, "
                  f"IS MDD={winner['max_drawdown']:.3%}, "
                  f"IS trades={int(winner['trade_count'])}", flush=True)

            # ---- 2) Run this promoted config out-of-sample ----
            promoted_params = {
                'annualized_target_volatility': winner.get('annualized_target_volatility', fixed_params.get('annualized_target_volatility')),
                'stop_loss_strategy':           winner.get('stop_loss_strategy',           fixed_params.get('stop_loss_strategy')),
                'rolling_atr_window':           winner.get('rolling_atr_window',           fixed_params.get('rolling_atr_window')),
                'atr_multiplier':               winner.get('atr_multiplier',               fixed_params.get('atr_multiplier')),
                'highest_high_window':          winner.get('highest_high_window',          fixed_params.get('highest_high_window')),
                'cooldown_counter_threshold':   winner.get('cooldown_counter_threshold',   fixed_params.get('cooldown_counter_threshold')),
            }
            p_os = fixed_params.copy()
            p_os.update(promoted_params)
            p_os = _clean_params(p_os)

            print(f"[WFA] Fold {fold_idx}: OS run (rank {int(winner['promotion_rank'])}) with params: {_fmt_params(p_os)}", flush=True)
            t1 = time.perf_counter()

            df_os = tf_fn(
                start_date=start_os - pd.Timedelta(days=warmup_days),
                end_date=end_os, ticker_list=ticker_list,
                **p_os,
                use_specific_start_date=True,
                signal_start_date=start_os,
            )
            df_os = df_os[df_os.index >= pd.Timestamp(start_os).date()]
            df_os = perf.calculate_asset_level_returns(df_os, end_os, ticker_list)

            os_port = perf_fn(
                df_os,
                strategy_daily_return_col='portfolio_daily_pct_returns',
                strategy_trade_count_col='count_of_positions',
                include_transaction_costs_and_fees=False,
                passive_trade_rate=0.05, annual_trading_days=365,
                transaction_cost_est=0.001
            )

            print(f"[WFA] Fold {fold_idx}: OS METRICS (rank {int(winner['promotion_rank'])}): "
                  f"Sharpe={os_port.get('annualized_sharpe_ratio',np.nan):.3f}, "
                  f"MDD={os_port.get('max_drawdown',np.nan):.3%}, "
                  f"trades={int(os_port.get('trade_count',0))} "
                  f"(elapsed {time.perf_counter()-t1:.1f}s)", flush=True)

            os_row = {
                'sampling_category':'out_sample',
                'start_date': start_os,
                'end_date': end_os,
                'is_score': float(winner['is_score']),
                'os_score': float(os_port['annualized_sharpe_ratio']),  # OS score = OS Sharpe
                'is_promoted': True,
                'promotion_rank': int(winner['promotion_rank']),
                'promotion_set_id': promotion_set_id,
                **{k: p_os.get(k, np.nan) for k in [
                    'annualized_target_volatility','stop_loss_strategy','atr_multiplier',
                    'rolling_atr_window','highest_high_window','cooldown_counter_threshold'
                ]},
                **os_port
            }
            if include_ticker_metrics:
                for t in ticker_list:
                    tmet = perf.calculate_risk_and_performance_metrics(
                        df_os,
                        strategy_daily_return_col=f'{t}_daily_pct_returns',
                        strategy_trade_count_col=f'{t}_position_count',
                        annual_trading_days=365,
                        include_transaction_costs_and_fees=False
                    )
                    for c in ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']:
                        os_row[f'{t}_{c}'] = tmet[c]

            df_performance.loc[len(df_performance)] = os_row

        print(f"[WFA] Fold {fold_idx}: complete. Rolling forward…", flush=True)
        # ---- roll forward ----
        start_is = (start_is + (OS_LEN if step_equals_os else pd.DateOffset(months=1))).date()

    print(f"\n[WFA] finished. folds={fold_idx}, total_rows={len(df_performance)} "
          f"(elapsed {time.perf_counter()-start_ts:.1f}s)", flush=True)

    return df_performance


In [58]:
fixed = dict(
    # your frozen prod config passed into tf_fn
    fast_mavg=20, slow_mavg=200, mavg_stepsize=8, mavg_z_score_window=126,
    entry_rolling_donchian_window=56, exit_rolling_donchian_window=28,
    use_donchian_exit_gate=False,
    ma_crossover_signal_weight=0.9, donchian_signal_weight=0.1, weighted_signal_ewm_window=4,
    rolling_r2_window=100, lower_r_sqr_limit=0.45, upper_r_sqr_limit=0.9,
    r2_smooth_window=3, r2_confirm_days=0, r2_strong_threshold=0.75,
    log_std_window=14, coef_of_variation_window=20, vol_of_vol_z_score_window=126, vol_of_vol_p_min=0.10,
    use_activation=False, tanh_activation_constant_dict=None,
    moving_avg_type='exponential', long_only=True, price_or_returns_calc='price',
    initial_capital=15000, rolling_cov_window=20, volatility_window=30,
    stop_loss_strategy='Chandelier', rolling_atr_window=14,
    atr_multiplier=2.25, highest_high_window=56, cooldown_counter_threshold=1,
    annualized_target_volatility=0.40,  # fallback if sweep doesn't set it
    transaction_cost_est=0.001, passive_trade_rate=0.05,
    notional_threshold_pct=0.10, rolling_sharpe_window=50, cash_buffer_percentage=0.10,
    annual_trading_days=365, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-07-31',
    # use_specific_start_date=True, signal_start_date='2022-04-01'
)
sweep = {"cooldown_counter_threshold": [0, 1, 2, 3, 5]}

df_cooldown = run_wfa(
    start_date="2022-04-01", end_date="2025-07-31",
    ticker_list=['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD'],
    is_months=18, os_months=3, step_equals_os=True, warmup_days=300,
    fixed_params=fixed, sweep_params=sweep,
    tf_fn=tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol,
    perf_fn=calculate_risk_and_performance_metrics,
    include_ticker_metrics=True
)


[WFA] init: IS=18m, OS=3m, STEP=OS_LEN, warmup=300d, tickers=5, grid_size=5, promote_top_k=1

[WFA] Fold 1: IS 2022-04-01 → 2023-09-30 | OS 2023-10-01 → 2023-12-31 | warmup=300d
[WFA] Fold 1: evaluating 5 config(s) in-sample…
[WFA] Fold 1 | IS cfg 1/5: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=0
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
[WFA] Fold 1 | IS cfg 1/5 METRICS: Sharpe=0.150, MDD=-13.103%, trades=58
[WFA] Fold 1 | IS cfg 2/5: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=1
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Tre

In [36]:
df_cooldown['vol_tracking_error'] = (np.abs(df_cooldown['annualized_std_dev'] - df_cooldown['annualized_target_volatility']) / df_cooldown['annualized_target_volatility'])

in_sample_cond = (df_cooldown.sampling_category == 'in_sample')
df_cooldown_is = df_cooldown[in_sample_cond].reset_index(drop=True)
df_cooldown_os = df_cooldown[~in_sample_cond].reset_index(drop=True)

In [60]:
df_cooldown.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Cooldown_Counter_Threshold_Performance-2022-04-01-2025-10-01.pickle')
# df_cooldown = pd.read_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Cooldown_Counter_Threshold_Performance-2022-04-01-2025-10-01.pickle')

In [62]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'annualized_std_dev':['median','mean','std'],
            'vol_tracking_error':['median','mean','std'],
            'trade_count':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}

In [64]:
df_cooldown.head()

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,promotion_rank,promotion_set_id,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,0.062030,0.150180,0.473397,0.386539,-0.131031,292 days,0.100365,0.556783,0.577903,58.0,0.4,Chandelier,2.25,14,56,0,-0.313040,NaN,False,NaN,None,-0.007595,-0.439854,0.266515,-0.127637,0.020862,-0.594587,0.130958,-0.036686,0.039333,-0.111741,0.646857,-0.022147,-0.030570,-0.926539,0.293654,-0.111915,0.008067,-1.666147,0.091952,-0.014681
1,in_sample,2022-04-01,2023-09-30,0.062030,0.150180,0.473397,0.386539,-0.131031,292 days,0.100365,0.556783,0.577903,58.0,0.4,Chandelier,2.25,14,56,1,-0.313040,NaN,False,NaN,None,-0.007595,-0.439854,0.266515,-0.127637,0.020862,-0.594587,0.130958,-0.036686,0.039333,-0.111741,0.646857,-0.022147,-0.030570,-0.926539,0.293654,-0.111915,0.008067,-1.666147,0.091952,-0.014681
2,in_sample,2022-04-01,2023-09-30,0.070289,0.198539,0.600943,0.394386,-0.116965,290 days,0.098540,0.616520,0.537808,56.0,0.4,Chandelier,2.25,14,56,2,0.321013,NaN,False,NaN,None,-0.010426,-0.465312,0.269127,-0.127637,0.020889,-0.594107,0.130937,-0.036686,0.039333,-0.111741,0.646857,-0.022147,-0.016029,-0.766532,0.290350,-0.091840,0.008067,-1.666147,0.091952,-0.014681
3,in_sample,2022-04-01,2023-09-30,0.077987,0.243459,0.666757,0.398218,-0.116965,290 days,0.098540,0.672333,0.501655,54.0,0.4,Chandelier,2.25,14,56,3,0.645442,NaN,False,NaN,None,-0.000565,-0.379679,0.270918,-0.127637,0.020889,-0.594107,0.130937,-0.036686,0.039333,-0.111741,0.646857,-0.022147,-0.016029,-0.766532,0.290350,-0.091840,0.008067,-1.666147,0.091952,-0.014681
4,in_sample,2022-04-01,2023-09-30,0.077095,0.238489,0.659127,0.400616,-0.116965,290 days,0.096715,0.667268,0.504882,54.0,0.4,Chandelier,2.25,14,56,5,0.640472,NaN,False,NaN,None,-0.000536,-0.382530,0.272094,-0.127637,0.020889,-0.594107,0.130937,-0.036686,0.039333,-0.111741,0.646857,-0.022147,-0.016029,-0.766532,0.290350,-0.091840,0.008067,-1.666147,0.091952,-0.014681


In [70]:
df_cooldown_is.groupby(['annualized_target_volatility','cooldown_counter_threshold']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                         median   
annualized_target_volatility cooldown_counter_threshold                           
0.4                          5                                         1.974660   
                             3                                         1.925466   
                             2                                         1.810631   
                             0                                         1.761367   
                             1                                         1.761367   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          5                           1.801299  0.723775   
                             3                           1.759239  0.702739   
                             2                           1.680900  0.689424   
                             0                           1.609636  0.686733   
                             1                           1.609636  0.686733   

                                                        annualized_return  \
                                                                   median   
annualized_target_volatility cooldown_counter_threshold                     
0.4                          5                                   0.571041   
                             3                                   0.562996   
                             2                                   0.523862   
                             0                                   0.515139   
                             1                                   0.515139   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          5                           0.574705  0.252616   
                             3                           0.564260  0.247231   
                             2                           0.537001  0.237688   
                             0                           0.513264  0.232470   
                             1                           0.513264  0.232470   

                                                        max_drawdown  \
                                                              median   
annualized_target_volatility cooldown_counter_threshold                
0.4                          5                             -0.118575   
                             3                             -0.125623   
                             2                             -0.151502   
                             0                             -0.175480   
                             1                             -0.175480   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          5                          -0.124728  0.026576   
                             3                          -0.133354  0.032749   
                             2                          -0.146423  0.037497   
                             0                          -0.169407  0.049690   
                             1                          -0.169407  0.049690   

                                                        annualized_std_dev  \
                                                                    median   
annualized_target_volatility cooldown_counter_threshold                      
0.4                          5                                  

In [68]:
df_cooldown_os.groupby(['annualized_target_volatility','cooldown_counter_threshold']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                         median   
annualized_target_volatility cooldown_counter_threshold                           
0.4                          3                                         5.219693   
                             5                                        -0.333129   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          3                           5.219693       NaN   
                             5                          -0.888401  3.068434   

                                                        annualized_return  \
                                                                   median   
annualized_target_volatility cooldown_counter_threshold                     
0.4                          3                                   6.512045   
                             5                                  -0.018583   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          3                           6.512045       NaN   
                             5                           0.178954  0.614757   

                                                        max_drawdown  \
                                                              median   
annualized_target_volatility cooldown_counter_threshold                
0.4                          3                             -0.061202   
                             5                             -0.076421   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          3                          -0.061202       NaN   
                             5                          -0.069327  0.028143   

                                                        annualized_std_dev  \
                                                                    median   
annualized_target_volatility cooldown_counter_threshold                      
0.4                          3                                    0.538290   
                             5                                    0.335879   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          3                           0.538290       NaN   
                             5                           0.303254  0.141983   

                                                        vol_tracking_error  \
                                                                    median   
annualized_target_volatility cooldown_counter_threshold                      
0.4                          3                                    0.345725   
                             5                                    0.184904   

                                                                           \
                                                             mean     std   
annualized_target_volatility cooldown_counter_threshold                     
0.4                          3                           0.345725     NaN   
                             5                           0.310520  0.2837   

                                                        trade_count        \
                                                             med

### Coodown Walk Forward Promoting Top 2 Winners to Out of Sample

In [22]:
fixed = dict(
    # your frozen prod config passed into tf_fn
    fast_mavg=20, slow_mavg=200, mavg_stepsize=8, mavg_z_score_window=126,
    entry_rolling_donchian_window=56, exit_rolling_donchian_window=28,
    use_donchian_exit_gate=False,
    ma_crossover_signal_weight=0.9, donchian_signal_weight=0.1, weighted_signal_ewm_window=4,
    rolling_r2_window=100, lower_r_sqr_limit=0.45, upper_r_sqr_limit=0.9,
    r2_smooth_window=3, r2_confirm_days=0, r2_strong_threshold=0.75,
    log_std_window=14, coef_of_variation_window=20, vol_of_vol_z_score_window=126, vol_of_vol_p_min=0.10,
    use_activation=False, tanh_activation_constant_dict=None,
    moving_avg_type='exponential', long_only=True, price_or_returns_calc='price',
    initial_capital=15000, rolling_cov_window=20, volatility_window=30,
    stop_loss_strategy='Chandelier', rolling_atr_window=14,
    atr_multiplier=2.25, highest_high_window=56, cooldown_counter_threshold=1,
    annualized_target_volatility=0.40,  # fallback if sweep doesn't set it
    transaction_cost_est=0.001, passive_trade_rate=0.05,
    notional_threshold_pct=0.10, rolling_sharpe_window=50, cash_buffer_percentage=0.10,
    annual_trading_days=365, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-07-31',
    # use_specific_start_date=True, signal_start_date='2022-04-01'
)
sweep = {"cooldown_counter_threshold": [0, 1, 2, 3, 5]}

df_cooldown_top_2 = run_wfa(
    start_date="2022-04-01", end_date="2025-07-31",
    ticker_list=['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD'],
    is_months=18, os_months=3, step_equals_os=True, warmup_days=300,
    fixed_params=fixed, sweep_params=sweep,
    tf_fn=tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol,
    perf_fn=calculate_risk_and_performance_metrics,
    include_ticker_metrics=True, promote_top_k=2
)


[WFA] init: IS=18m, OS=3m, STEP=OS_LEN, warmup=300d, tickers=5, grid_size=5, promote_top_k=2

[WFA] Fold 1: IS 2022-04-01 → 2023-09-30 | OS 2023-10-01 → 2023-12-31 | warmup=300d
[WFA] Fold 1: evaluating 5 config(s) in-sample…
[WFA] Fold 1 | IS cfg 1/5: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=0
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
[WFA] Fold 1 | IS cfg 1/5 METRICS: Sharpe=0.150, MDD=-13.103%, trades=58
[WFA] Fold 1 | IS cfg 2/5: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=1
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Tre

In [78]:
df_cooldown_top_2['vol_tracking_error'] = (np.abs(df_cooldown_top_2['annualized_std_dev'] - df_cooldown_top_2['annualized_target_volatility']) / df_cooldown_top_2['annualized_target_volatility'])

df_cooldown_top_2 = df_cooldown_top_2.replace([np.inf, -np.inf], np.nan)
in_sample_cond = (df_cooldown_top_2.sampling_category == 'in_sample')
df_cooldown_top_2_is = df_cooldown_top_2[in_sample_cond].reset_index(drop=True)
df_cooldown_top_2_os = df_cooldown_top_2[~in_sample_cond].reset_index(drop=True)

In [86]:
df_cooldown_top_2.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Cooldown_Counter_Threshold_Performance_Top_2-2022-04-01-2025-10-01.pickle')
# df_cooldown = pd.read_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Cooldown_Counter_Threshold_Performance-2022-04-01-2025-10-01.pickle')

In [80]:
df_cooldown_top_2_os

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,promotion_rank,promotion_set_id,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,out_sample,2023-10-01,2023-12-31,6.512045,5.219693,106.402715,0.538290,-0.061202,22 days,0.489130,2.682986,0.008667,37.0,0.4,Chandelier,2.25,14,56,3,0.645442,5.219693,True,1.0,fold1_2023-10-01,0.171784,0.741041,0.239090,-0.042900,-0.031087,-2.516566,0.067399,-0.017042,0.162977,1.011857,0.201753,-0.025926,1.317091,5.164364,0.176486,-0.022565,2.003888,4.303568,0.520702,-0.017236,0.345725
1,out_sample,2023-10-01,2023-12-31,6.842869,5.365930,111.808164,0.534997,-0.061202,22 days,0.500000,2.756856,0.007052,37.0,0.4,Chandelier,2.25,14,56,5,0.640472,5.365930,True,2.0,fold1_2023-10-01,0.191980,0.846902,0.243449,-0.042900,-0.005094,-1.712277,0.074394,-0.010595,0.162222,1.008508,0.201144,-0.025662,1.317573,5.168647,0.176379,-0.022484,2.003888,4.303568,0.520702,-0.017236,0.337492
2,out_sample,2024-01-01,2024-03-31,1.376439,3.099172,13.328122,0.454115,-0.103273,37 days,0.362637,1.635773,0.105380,19.0,0.4,Chandelier,2.25,14,56,5,2.276479,3.099172,True,1.0,fold2_2024-01-01,0.257527,1.901412,0.127324,-0.023014,1.168696,3.365152,0.303620,-0.079094,-0.026717,-7.974690,0.005347,-0.006729,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.135287
3,out_sample,2024-01-01,2024-03-31,1.330826,2.975636,11.609181,0.448797,-0.114636,37 days,0.362637,1.572387,0.119370,19.0,0.4,Chandelier,2.25,14,56,3,2.240116,2.975636,True,2.0,fold2_2024-01-01,0.195930,1.342947,0.130734,-0.035184,1.244228,3.490112,0.300090,-0.079094,-0.026717,-7.974690,0.005347,-0.006729,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.121992
4,out_sample,2024-04-01,2024-06-30,-0.179848,-2.859152,-3.274038,0.145568,-0.054932,90 days,0.032967,-1.141434,0.256717,6.0,0.4,Chandelier,2.25,14,56,5,2.868666,-2.859152,True,1.0,fold3_2024-04-01,-0.163377,-2.452762,0.275429,-0.043499,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.636081
5,out_sample,2024-04-01,2024-06-30,-0.165647,-2.547227,-3.255767,0.150959,-0.050878,90 days,0.054945,-0.997191,0.321345,7.0,0.4,Chandelier,2.25,14,56,3,2.709737,-2.547227,True,2.0,fold3_2024-04-01,-0.143531,-2.157884,0.257888,-0.037892,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.622603
6,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,2.25,14,56,5,2.829540,0.373578,True,1.0,fold4_2024-07-01,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
7,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,2.25,14,56,3,2.689252,0.373578,True,2.0,fold4_2024-07-01,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
8,out_sample,2024-10-01,2024-12-31,0.211057,0.771742,2.596985,0.365566,-0.081270,40 days,0.271739,0.501984,0.616891,26.0,0.4,Chandelier,2.2

In [82]:
df_cooldown_top_2_is.groupby(['annualized_target_volatility','cooldown_counter_threshold']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                         median   
annualized_target_volatility cooldown_counter_threshold                           
0.4                          5                                         1.974660   
                             3                                         1.925466   
                             2                                         1.810631   
                             0                                         1.761367   
                             1                                         1.761367   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          5                           1.801299  0.723775   
                             3                           1.759239  0.702739   
                             2                           1.680900  0.689424   
                             0                           1.609636  0.686733   
                             1                           1.609636  0.686733   

                                                        annualized_return  \
                                                                   median   
annualized_target_volatility cooldown_counter_threshold                     
0.4                          5                                   0.571041   
                             3                                   0.562996   
                             2                                   0.523862   
                             0                                   0.515139   
                             1                                   0.515139   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          5                           0.574705  0.252616   
                             3                           0.564260  0.247231   
                             2                           0.537001  0.237688   
                             0                           0.513264  0.232470   
                             1                           0.513264  0.232470   

                                                        max_drawdown  \
                                                              median   
annualized_target_volatility cooldown_counter_threshold                
0.4                          5                             -0.118575   
                             3                             -0.125623   
                             2                             -0.151502   
                             0                             -0.175480   
                             1                             -0.175480   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          5                          -0.124728  0.026576   
                             3                          -0.133354  0.032749   
                             2                          -0.146423  0.037497   
                             0                          -0.169407  0.049690   
                             1                          -0.169407  0.049690   

                                                        annualized_std_dev  \
                                                                    median   
annualized_target_volatility cooldown_counter_threshold                      
0.4                          5                                  

In [84]:
df_cooldown_top_2_os.groupby(['annualized_target_volatility','cooldown_counter_threshold']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                         median   
annualized_target_volatility cooldown_counter_threshold                           
0.4                          3                                         0.373578   
                             5                                         0.373578   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          3                           0.007348  3.339299   
                             5                           0.005075  3.665265   

                                                        annualized_return  \
                                                                   median   
annualized_target_volatility cooldown_counter_threshold                     
0.4                          3                                   0.085207   
                             5                                   0.085207   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          3                           1.056622  2.468760   
                             5                           1.130942  2.580485   

                                                        max_drawdown  \
                                                              median   
annualized_target_volatility cooldown_counter_threshold                
0.4                          3                             -0.067739   
                             5                             -0.071572   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          3                          -0.073227  0.034353   
                             5                          -0.068166  0.025874   

                                                        annualized_std_dev  \
                                                                    median   
annualized_target_volatility cooldown_counter_threshold                      
0.4                          3                                    0.355193   
                             5                                    0.365566   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          3                           0.329467  0.157224   
                             5                           0.336360  0.156433   

                                                        vol_tracking_error  \
                                                                    median   
annualized_target_volatility cooldown_counter_threshold                      
0.4                          3                                    0.337081   
                             5                                    0.234520   

                                                                             \
                                                             mean       std   
annualized_target_volatility cooldown_counter_threshold                       
0.4                          3                           0.330159  0.252190   
                             5                           0.314373  0.259182   

                                                        trade_count  \
                                                            

In [92]:
df_cooldown_top_2_os.groupby(['annualized_target_volatility','start_date','cooldown_counter_threshold']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                    median   
annualized_target_volatility start_date cooldown_counter_threshold                           
0.4                          2023-10-01 5                                         5.365930   
                                        3                                         5.219693   
                             2024-01-01 5                                         3.099172   
                                        3                                         2.975636   
                             2024-10-01 5                                         0.771742   
                                        3                                         0.592751   
                             2024-07-01 3                                         0.373578   
                                        5                                         0.373578   
                             2025-04-01 5                                        -1.039836   
                                        3                                        -2.211323   
                             2024-04-01 3                                        -2.547227   
                                        5                                        -2.859152   
                             2025-01-01 3                                        -4.351674   
                                        5                                        -5.675910   

                                                                              \
                                                                        mean   
annualized_target_volatility start_date cooldown_counter_threshold             
0.4                          2023-10-01 5                           5.365930   
                                        3                           5.219693   
                             2024-01-01 5                           3.099172   
                                        3                           2.975636   
                             2024-10-01 5                           0.771742   
                                        3                           0.592751   
                             2024-07-01 3                           0.373578   
                                        5                           0.373578   
                             2025-04-01 5                          -1.039836   
                                        3                          -2.211323   
                             2024-04-01 3                          -2.547227   
                                        5                          -2.859152   
                             2025-01-01 3                          -4.351674   
                                        5                          -5.675910   

                                                                        \
                                                                   std   
annualized_target_volatility start_date cooldown_counter_threshold       
0.4                          2023-10-01 5                          NaN   
                                        3                          NaN   
                             2024-01-01 5                          NaN   
                                        3                          NaN   
                             2024-10-01 5                          NaN   
                                        3                          NaN   
                             2024-07-01 3                          NaN   
                                        5                          NaN   
                             2025-04-01 5                          NaN   
                                        3                          NaN   
                             2024-04-01 3                          NaN   
                                        5                          NaN 

## ATR Multiplier Walk Forward Analysis
### Lockdown Cooldown Counter Threshold to 5

In [99]:
fixed = dict(
    # your frozen prod config passed into tf_fn
    fast_mavg=20, slow_mavg=200, mavg_stepsize=8, mavg_z_score_window=126,
    entry_rolling_donchian_window=56, exit_rolling_donchian_window=28,
    use_donchian_exit_gate=False,
    ma_crossover_signal_weight=0.9, donchian_signal_weight=0.1, weighted_signal_ewm_window=4,
    rolling_r2_window=100, lower_r_sqr_limit=0.45, upper_r_sqr_limit=0.9,
    r2_smooth_window=3, r2_confirm_days=0, r2_strong_threshold=0.75,
    log_std_window=14, coef_of_variation_window=20, vol_of_vol_z_score_window=126, vol_of_vol_p_min=0.10,
    use_activation=False, tanh_activation_constant_dict=None,
    moving_avg_type='exponential', long_only=True, price_or_returns_calc='price',
    initial_capital=15000, rolling_cov_window=20, volatility_window=30,
    stop_loss_strategy='Chandelier', rolling_atr_window=14,
    atr_multiplier=2.25, highest_high_window=56, cooldown_counter_threshold=5,
    annualized_target_volatility=0.40,  # fallback if sweep doesn't set it
    transaction_cost_est=0.001, passive_trade_rate=0.05,
    notional_threshold_pct=0.10, rolling_sharpe_window=50, cash_buffer_percentage=0.10,
    annual_trading_days=365, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-07-31',
    # use_specific_start_date=True, signal_start_date='2022-04-01'
)
sweep = {"atr_multiplier": [1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0]}

df_atr_multiplier = run_wfa(
    start_date="2022-04-01", end_date="2025-07-31",
    ticker_list=['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD'],
    is_months=18, os_months=3, step_equals_os=True, warmup_days=300,
    fixed_params=fixed, sweep_params=sweep,
    tf_fn=tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol,
    perf_fn=calculate_risk_and_performance_metrics,
    include_ticker_metrics=True, promote_top_k=2
)


[WFA] init: IS=18m, OS=3m, STEP=OS_LEN, warmup=300d, tickers=5, grid_size=7, promote_top_k=2

[WFA] Fold 1: IS 2022-04-01 → 2023-09-30 | OS 2023-10-01 → 2023-12-31 | warmup=300d
[WFA] Fold 1: evaluating 7 config(s) in-sample…
[WFA] Fold 1 | IS cfg 1/7: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=1.5, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=5
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
[WFA] Fold 1 | IS cfg 1/7 METRICS: Sharpe=0.104, MDD=-7.944%, trades=53
[WFA] Fold 1 | IS cfg 2/7: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=1.75, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=5
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend

In [101]:
df_atr_multiplier['vol_tracking_error'] = (np.abs(df_atr_multiplier['annualized_std_dev'] - df_atr_multiplier['annualized_target_volatility']) / df_atr_multiplier['annualized_target_volatility'])

df_atr_multiplier = df_atr_multiplier.replace([np.inf, -np.inf], np.nan)
in_sample_cond = (df_atr_multiplier.sampling_category == 'in_sample')
df_atr_multiplier_is = df_atr_multiplier[in_sample_cond].reset_index(drop=True)
df_atr_multiplier_os = df_atr_multiplier[~in_sample_cond].reset_index(drop=True)

In [103]:
df_atr_multiplier_os

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,promotion_rank,promotion_set_id,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,out_sample,2023-10-01,2023-12-31,3.642604,4.969109,55.973267,0.511053,-0.065078,25 days,0.434783,2.574009,0.011668,37.0,0.4,Chandelier,1.75,14,56,5,0.812362,4.969109,True,1.0,fold1_2023-10-01,0.000273,-0.479968,0.169421,-0.037761,-0.004463,-2.330565,0.065402,-0.006784,0.123108,0.852048,0.195468,-0.025211,0.689374,5.469564,0.126109,-0.011009,1.932473,4.207742,0.534999,-0.017236,0.277631
1,out_sample,2023-10-01,2023-12-31,3.518351,4.932626,55.904004,0.463485,-0.062936,25 days,0.391304,2.556589,0.012227,40.0,0.4,Chandelier,1.50,14,56,5,0.511649,4.932626,True,2.0,fold1_2023-10-01,-0.047470,-1.073847,0.185771,-0.039915,-0.030102,-4.150370,0.056040,-0.012643,0.115819,0.780440,0.232286,-0.025211,0.691981,4.942565,0.148547,-0.013665,1.932473,4.207742,0.534999,-0.017236,0.158712
2,out_sample,2024-01-01,2024-03-31,1.424936,3.328357,18.103439,0.416935,-0.078711,45 days,0.307692,1.754949,0.082671,21.0,0.4,Chandelier,1.75,14,56,5,2.002258,3.328357,True,1.0,fold2_2024-01-01,0.147546,1.111411,0.118215,-0.022401,1.174727,3.379370,0.312851,-0.079094,-0.014234,-8.831106,NaN,-0.003568,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.042339
3,out_sample,2024-01-01,2024-03-31,1.376439,3.099172,13.328122,0.454115,-0.103273,37 days,0.362637,1.635773,0.105380,19.0,0.4,Chandelier,2.25,14,56,5,1.928449,3.099172,True,2.0,fold2_2024-01-01,0.257527,1.901412,0.127324,-0.023014,1.168696,3.365152,0.303620,-0.079094,-0.026717,-7.974690,0.005347,-0.006729,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.135287
4,out_sample,2024-04-01,2024-06-30,-0.179848,-2.859152,-3.274038,0.145568,-0.054932,90 days,0.032967,-1.141434,0.256717,6.0,0.4,Chandelier,2.25,14,56,5,2.577733,-2.859152,True,1.0,fold3_2024-04-01,-0.163377,-2.452762,0.275429,-0.043499,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.636081
5,out_sample,2024-04-01,2024-06-30,-0.117676,-3.041687,-3.829211,0.078844,-0.030731,90 days,0.021978,-1.088851,0.279127,4.0,0.4,Chandelier,1.75,14,56,5,2.519190,-3.041687,True,2.0,fold3_2024-04-01,-0.140669,-2.722909,0.281513,-0.037091,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.802890
6,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,3.00,14,56,5,2.507002,0.373578,True,1.0,fold4_2024-07-01,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
7,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,1.75,14,56,5,2.496172,0.373578,True,2.0,fold4_2024-07-01,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
8,out_sample,2024-10-01,2024-12-31,0.255413,0.983394,3.466754,0.386486,-0.073675,38 days,0.228261,0.615028,0.540071,26.0,0.4,Chandelier,1.75,14

In [105]:
df_atr_multiplier.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/ATR_Multiplier_Performance_Top_2-2022-04-01-2025-10-01.pickle')
# df_atr_multiplier = pd.read_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Cooldown_Counter_Threshold_Performance-2022-04-01-2025-10-01.pickle')

In [109]:
df_atr_multiplier_is.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                        median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier                           
0.4                          5                          1.50                          1.998685   
                                                        2.25                          1.974660   
                                                        1.75                          1.966058   
                                                        3.00                          1.901079   
                                                        2.00                          1.834697   
                                                        2.50                          1.786197   
                                                        2.75                          1.669438   

                                                                                  \
                                                                            mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier             
0.4                          5                          1.50            1.670431   
                                                        2.25            1.801299   
                                                        1.75            1.675898   
                                                        3.00            1.783678   
                                                        2.00            1.474042   
                                                        2.50            1.676678   
                                                        2.75            1.579124   

                                                                                  \
                                                                             std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier             
0.4                          5                          1.50            0.736336   
                                                        2.25            0.723775   
                                                        1.75            0.679776   
                                                        3.00            0.697268   
                                                        2.00            0.742914   
                                                        2.50            0.708883   
                                                        2.75            0.741117   

                                                                       annualized_return  \
                                                                                  median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier                     
0.4                          5                          1.50                    0.522750   
                                                        2.25                    0.571041   
                                                        1.75                    0.487643   
                                                        3.00                    0.626226   
                                                        2.00                    0.486541   
                                                        2.50                    0.555327   
                                                        2.75                    0.538866   

                                                                                  \
                                                                            mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier             
0.4                          5                          1.50            0.422198   
                                                        2.25            0.574705   
                                                        1.75       

In [111]:
df_atr_multiplier_os.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                        median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier                           
0.4                          5                          1.75                          0.678486   
                                                        3.00                          0.303157   
                                                        2.25                          0.120010   
                                                        1.50                         -2.430536   

                                                                                  \
                                                                            mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier             
0.4                          5                          1.75            0.821241   
                                                        3.00           -0.689637   
                                                        2.25            0.120010   
                                                        1.50           -0.807651   

                                                                                  \
                                                                             std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier             
0.4                          5                          1.75            2.997403   
                                                        3.00            1.780904   
                                                        2.25            4.213171   
                                                        1.50            5.125302   

                                                                       annualized_return  \
                                                                                  median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier                     
0.4                          5                          1.75                    0.170310   
                                                        3.00                    0.085207   
                                                        2.25                    0.598295   
                                                        1.50                   -0.130303   

                                                                                  \
                                                                            mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier             
0.4                          5                          1.75            0.857796   
                                                        3.00           -0.035066   
                                                        2.25            0.598295   
                                                        1.50            1.060897   

                                                                                  \
                                                                             std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier             
0.4                          5                          1.75            1.482641   
                                                        3.00            0.218696   
                                                        2.25            1.100461   
                                                        1.50            2.128549   

                                                                       max_drawdown  \
                                                                             median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier                
0.4                          5                          1.75              -0.068995   
                                   

## ATR Window Walk Forward Analysis
### Lockdown ATR Muldiplier to 1.75

In [116]:
fixed = dict(
    # your frozen prod config passed into tf_fn
    fast_mavg=20, slow_mavg=200, mavg_stepsize=8, mavg_z_score_window=126,
    entry_rolling_donchian_window=56, exit_rolling_donchian_window=28,
    use_donchian_exit_gate=False,
    ma_crossover_signal_weight=0.9, donchian_signal_weight=0.1, weighted_signal_ewm_window=4,
    rolling_r2_window=100, lower_r_sqr_limit=0.45, upper_r_sqr_limit=0.9,
    r2_smooth_window=3, r2_confirm_days=0, r2_strong_threshold=0.75,
    log_std_window=14, coef_of_variation_window=20, vol_of_vol_z_score_window=126, vol_of_vol_p_min=0.10,
    use_activation=False, tanh_activation_constant_dict=None,
    moving_avg_type='exponential', long_only=True, price_or_returns_calc='price',
    initial_capital=15000, rolling_cov_window=20, volatility_window=30,
    stop_loss_strategy='Chandelier', rolling_atr_window=14,
    atr_multiplier=1.75, highest_high_window=56, cooldown_counter_threshold=5,
    annualized_target_volatility=0.40,  # fallback if sweep doesn't set it
    transaction_cost_est=0.001, passive_trade_rate=0.05,
    notional_threshold_pct=0.10, rolling_sharpe_window=50, cash_buffer_percentage=0.10,
    annual_trading_days=365, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-07-31',
    # use_specific_start_date=True, signal_start_date='2022-04-01'
)
sweep = {"rolling_atr_window": [14, 20, 22, 25, 30, 40]}

df_rolling_atr_window = run_wfa(
    start_date="2022-04-01", end_date="2025-07-31",
    ticker_list=['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD'],
    is_months=18, os_months=3, step_equals_os=True, warmup_days=300,
    fixed_params=fixed, sweep_params=sweep,
    tf_fn=tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol,
    perf_fn=calculate_risk_and_performance_metrics,
    include_ticker_metrics=True, promote_top_k=2
)


[WFA] init: IS=18m, OS=3m, STEP=OS_LEN, warmup=300d, tickers=5, grid_size=6, promote_top_k=2

[WFA] Fold 1: IS 2022-04-01 → 2023-09-30 | OS 2023-10-01 → 2023-12-31 | warmup=300d
[WFA] Fold 1: evaluating 6 config(s) in-sample…
[WFA] Fold 1 | IS cfg 1/6: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=1.75, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=5
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
[WFA] Fold 1 | IS cfg 1/6 METRICS: Sharpe=0.217, MDD=-9.000%, trades=49
[WFA] Fold 1 | IS cfg 2/6: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=1.75, rolling_atr_window=20, highest_high_window=56, cooldown_counter_threshold=5
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Tren

In [118]:
df_rolling_atr_window['vol_tracking_error'] = (np.abs(df_rolling_atr_window['annualized_std_dev'] - df_rolling_atr_window['annualized_target_volatility']) / df_rolling_atr_window['annualized_target_volatility'])

df_rolling_atr_window = df_rolling_atr_window.replace([np.inf, -np.inf], np.nan)
in_sample_cond = (df_rolling_atr_window.sampling_category == 'in_sample')
df_rolling_atr_window_is = df_rolling_atr_window[in_sample_cond].reset_index(drop=True)
df_rolling_atr_window_os = df_rolling_atr_window[~in_sample_cond].reset_index(drop=True)

In [120]:
df_rolling_atr_window.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Rolling_ATR_Widnow_Performance_Top_2-2022-04-01-2025-10-01.pickle')
# df_atr_multiplier = pd.read_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Cooldown_Counter_Threshold_Performance-2022-04-01-2025-10-01.pickle')

In [122]:
df_rolling_atr_window.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier','rolling_atr_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                                           median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window                           
0.4                          5                          1.75           22                                2.087397   
                                                                       20                                2.029717   
                                                                       14                                1.973919   
                                                                       40                                1.958528   
                                                                       25                                1.726433   
                                                                       30                                1.444641   

                                                                                                     \
                                                                                               mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window             
0.4                          5                          1.75           22                  1.715680   
                                                                       20                  1.603156   
                                                                       14                  2.087550   
                                                                       40                  1.335712   
                                                                       25                  0.512024   
                                                                       30                  0.276366   

                                                                                                     \
                                                                                                std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window             
0.4                          5                          1.75           22                  0.769502   
                                                                       20                  2.004059   
                                                                       14                  1.323532   
                                                                       40                  1.857586   
                                                                       25                  2.765018   
                                                                       30                  2.900482   

                                                                                          annualized_return  \
                                                                                                     median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window                     
0.4                          5                          1.75           22                          0.529535   
                                                                       20                          0.512648   
                                                                       14                          0.491514   
                                                                       40                          0.509382   
                                                                       25                          0.415933   
                                                                       30                          0.334386   

                                                                                                     \
                                                                             

In [124]:
df_rolling_atr_window_is.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier','rolling_atr_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                                           median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window                           
0.4                          5                          1.75           25                                2.098600   
                                                                       22                                2.087397   
                                                                       30                                2.079322   
                                                                       20                                2.029717   
                                                                       14                                1.966058   
                                                                       40                                1.958528   

                                                                                                     \
                                                                                               mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window             
0.4                          5                          1.75           25                  1.735881   
                                                                       22                  1.715680   
                                                                       30                  1.717571   
                                                                       20                  1.745133   
                                                                       14                  1.675898   
                                                                       40                  1.673087   

                                                                                                     \
                                                                                                std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window             
0.4                          5                          1.75           25                  0.770037   
                                                                       22                  0.769502   
                                                                       30                  0.752010   
                                                                       20                  0.690231   
                                                                       14                  0.679776   
                                                                       40                  0.723152   

                                                                                          annualized_return  \
                                                                                                     median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window                     
0.4                          5                          1.75           25                          0.535452   
                                                                       22                          0.529535   
                                                                       30                          0.505415   
                                                                       20                          0.512648   
                                                                       14                          0.487643   
                                                                       40                          0.509382   

                                                                                                     \
                                                                             

In [126]:
df_rolling_atr_window_os.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier','rolling_atr_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                                           median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window                           
0.4                          5                          1.75           14                                4.969109   
                                                                       20                                1.876145   
                                                                       40                                0.154897   
                                                                       30                               -1.389629   
                                                                       25                               -1.967154   

                                                                                                     \
                                                                                               mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window             
0.4                          5                          1.75           14                  4.969109   
                                                                       20                  1.354696   
                                                                       40                  0.154897   
                                                                       30                 -2.245745   
                                                                       25                 -2.343644   

                                                                                                     \
                                                                                                std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window             
0.4                          5                          1.75           14                       NaN   
                                                                       20                  3.507895   
                                                                       40                  4.569673   
                                                                       30                  3.685611   
                                                                       25                  3.892357   

                                                                                          annualized_return  \
                                                                                                     median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window                     
0.4                          5                          1.75           14                          3.642604   
                                                                       20                          0.755328   
                                                                       40                          0.570676   
                                                                       30                         -0.026045   
                                                                       25                         -0.169960   

                                                                                                     \
                                                                                               mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window             
0.4                          5                          1.75           14                  3.642604   
                                                                       20                  1.193344   
                                                                       40                  0.570676

In [128]:
df_rolling_atr_window_os

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,promotion_rank,promotion_set_id,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,out_sample,2023-10-01,2023-12-31,3.393979,4.790073,47.200647,0.498424,-0.071905,25 days,0.402174,2.484120,0.014817,36.0,0.4,Chandelier,1.75,20,56,5,0.827714,4.790073,True,1.0,fold1_2023-10-01,-0.081530,-1.470627,0.168078,-0.039915,-0.004522,-2.332985,0.065406,-0.006784,0.122786,0.849293,0.195297,-0.025211,0.719559,5.673582,0.124220,-0.010666,1.932473,4.207742,0.534999,-0.017236,0.246059
1,out_sample,2023-10-01,2023-12-31,3.642604,4.969109,55.973267,0.511053,-0.065078,25 days,0.434783,2.574009,0.011668,37.0,0.4,Chandelier,1.75,14,56,5,0.374605,4.969109,True,2.0,fold1_2023-10-01,0.000273,-0.479968,0.169421,-0.037761,-0.004463,-2.330565,0.065402,-0.006784,0.123108,0.852048,0.195468,-0.025211,0.689374,5.469564,0.126109,-0.011009,1.932473,4.207742,0.534999,-0.017236,0.277631
2,out_sample,2024-01-01,2024-03-31,1.425450,3.378711,19.339829,0.400575,-0.073705,45 days,0.285714,1.781590,0.078189,22.0,0.4,Chandelier,1.75,20,56,5,2.135133,3.378711,True,1.0,fold2_2024-01-01,0.145278,1.120956,0.127998,-0.022401,1.174401,3.383170,0.326434,-0.079332,-0.014234,-8.831106,NaN,-0.003568,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.001438
3,out_sample,2024-01-01,2024-03-31,1.403439,3.386144,20.556965,0.396043,-0.068271,45 days,0.274725,1.786614,0.077367,21.0,0.4,Chandelier,1.75,40,56,5,1.594878,3.386144,True,2.0,fold2_2024-01-01,0.079788,0.410058,0.121657,-0.022401,1.414812,3.976729,0.316801,-0.055052,-0.014234,-8.831106,NaN,-0.003568,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.009892
4,out_sample,2024-04-01,2024-06-30,-0.137297,-3.152835,-3.797936,0.109341,-0.036150,90 days,0.021978,-1.179443,0.241330,4.0,0.4,Chandelier,1.75,30,56,5,2.501679,-3.152835,True,1.0,fold3_2024-04-01,-0.140669,-2.722909,0.281513,-0.037091,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.726647
5,out_sample,2024-04-01,2024-06-30,-0.131261,-3.123579,-3.807588,0.099922,-0.034474,90 days,0.021978,-1.154181,0.251482,4.0,0.4,Chandelier,1.75,20,56,5,2.478819,-3.123579,True,2.0,fold3_2024-04-01,-0.140669,-2.722909,0.281513,-0.037091,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.750194
6,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,1.75,30,56,5,2.661234,0.373578,True,1.0,fold4_2024-07-01,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
7,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,1.75,20,56,5,2.457904,0.373578,True,2.0,fold4_2024-07-01,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
8,out_sample,2024-10-01,2024-12-31,0.219510,0.873165,3.217543,0.365693,-0.068223,39 days,0.195652,0.565994,0.572791,23.0,0.4,Chandelier,1.75,30,56,5

## Highest High Window Walk Forward Analysis
### Lockdown Rolling ATR Window to 20

In [132]:
fixed = dict(
    # your frozen prod config passed into tf_fn
    fast_mavg=20, slow_mavg=200, mavg_stepsize=8, mavg_z_score_window=126,
    entry_rolling_donchian_window=56, exit_rolling_donchian_window=28,
    use_donchian_exit_gate=False,
    ma_crossover_signal_weight=0.9, donchian_signal_weight=0.1, weighted_signal_ewm_window=4,
    rolling_r2_window=100, lower_r_sqr_limit=0.45, upper_r_sqr_limit=0.9,
    r2_smooth_window=3, r2_confirm_days=0, r2_strong_threshold=0.75,
    log_std_window=14, coef_of_variation_window=20, vol_of_vol_z_score_window=126, vol_of_vol_p_min=0.10,
    use_activation=False, tanh_activation_constant_dict=None,
    moving_avg_type='exponential', long_only=True, price_or_returns_calc='price',
    initial_capital=15000, rolling_cov_window=20, volatility_window=30,
    stop_loss_strategy='Chandelier', rolling_atr_window=20,
    atr_multiplier=1.75, highest_high_window=56, cooldown_counter_threshold=5,
    annualized_target_volatility=0.40,  # fallback if sweep doesn't set it
    transaction_cost_est=0.001, passive_trade_rate=0.05,
    notional_threshold_pct=0.10, rolling_sharpe_window=50, cash_buffer_percentage=0.10,
    annual_trading_days=365, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-07-31',
    # use_specific_start_date=True, signal_start_date='2022-04-01'
)
sweep = {"highest_high_window": [20, 25, 30, 35, 40, 45, 50, 56, 60, 65]}

df_highest_high_window = run_wfa(
    start_date="2022-04-01", end_date="2025-07-31",
    ticker_list=['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD'],
    is_months=18, os_months=3, step_equals_os=True, warmup_days=300,
    fixed_params=fixed, sweep_params=sweep,
    tf_fn=tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol,
    perf_fn=calculate_risk_and_performance_metrics,
    include_ticker_metrics=True, promote_top_k=2
)


[WFA] init: IS=18m, OS=3m, STEP=OS_LEN, warmup=300d, tickers=5, grid_size=10, promote_top_k=2

[WFA] Fold 1: IS 2022-04-01 → 2023-09-30 | OS 2023-10-01 → 2023-12-31 | warmup=300d
[WFA] Fold 1: evaluating 10 config(s) in-sample…
[WFA] Fold 1 | IS cfg 1/10: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=1.75, rolling_atr_window=20, highest_high_window=20, cooldown_counter_threshold=5
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
[WFA] Fold 1 | IS cfg 1/10 METRICS: Sharpe=0.432, MDD=-17.761%, trades=90
[WFA] Fold 1 | IS cfg 2/10: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=1.75, rolling_atr_window=20, highest_high_window=25, cooldown_counter_threshold=5
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjuste

In [134]:
df_highest_high_window['vol_tracking_error'] = (np.abs(df_highest_high_window['annualized_std_dev'] - df_highest_high_window['annualized_target_volatility']) / df_highest_high_window['annualized_target_volatility'])

df_highest_high_window = df_highest_high_window.replace([np.inf, -np.inf], np.nan)
in_sample_cond = (df_highest_high_window.sampling_category == 'in_sample')
df_highest_high_window_is = df_highest_high_window[in_sample_cond].reset_index(drop=True)
df_highest_high_window_os = df_highest_high_window[~in_sample_cond].reset_index(drop=True)

In [136]:
df_highest_high_window.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Highest_High_Window_Performance_Top_2-2022-04-01-2025-10-01.pickle')
# df_atr_multiplier = pd.read_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Cooldown_Counter_Threshold_Performance-2022-04-01-2025-10-01.pickle')

In [140]:
df_highest_high_window.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier','rolling_atr_window','highest_high_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                                                               median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                           
0.4                          5                          1.75           20                 25                                 2.266189   
                                                                                          20                                 2.262713   
                                                                                          45                                 2.227795   
                                                                                          30                                 2.198436   
                                                                                          35                                 2.192578   
                                                                                          40                                 2.100413   
                                                                                          56                                 2.061562   
                                                                                          65                                 1.980418   
                                                                                          60                                 1.964311   
                                                                                          50                                 1.687307   

                                                                                                                         \
                                                                                                                   mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 25                   1.950617   
                                                                                          20                   1.985184   
                                                                                          45                   1.872569   
                                                                                          30                   1.817833   
                                                                                          35                   1.782278   
                                                                                          40                   0.884505   
                                                                                          56                   2.125751   
                                                                                          65                   1.670900   
                                                                                          60                   1.273964   
                                                                                          50                   0.525079   

                                                                                                                         \
                                                                                                                    std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 25                   0.737804   
                                                                                          20                   0.745948   
                                                                                          45                   0.

In [142]:
df_highest_high_window_is.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier','rolling_atr_window','highest_high_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                                                               median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                           
0.4                          5                          1.75           20                 25                                 2.266189   
                                                                                          20                                 2.262713   
                                                                                          40                                 2.227795   
                                                                                          45                                 2.227795   
                                                                                          50                                 2.219298   
                                                                                          30                                 2.198436   
                                                                                          35                                 2.192578   
                                                                                          56                                 2.029717   
                                                                                          60                                 2.022770   
                                                                                          65                                 1.923954   

                                                                                                                         \
                                                                                                                   mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 25                   1.950617   
                                                                                          20                   1.985184   
                                                                                          40                   1.891747   
                                                                                          45                   1.872569   
                                                                                          50                   1.891850   
                                                                                          30                   1.817833   
                                                                                          35                   1.782278   
                                                                                          56                   1.745133   
                                                                                          60                   1.731710   
                                                                                          65                   1.666257   

                                                                                                                         \
                                                                                                                    std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 25                   0.737804   
                                                                                          20                   0.745948   
                                                                                          40                   0.

In [144]:
df_highest_high_window_os.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier','rolling_atr_window','highest_high_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                                                               median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                           
0.4                          5                          1.75           20                 56                                 4.790073   
                                                                                          65                                 3.378711   
                                                                                          60                                -0.328148   
                                                                                          50                                -0.797226   
                                                                                          40                                -2.640839   

                                                                                                                         \
                                                                                                                   mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 56                   4.790073   
                                                                                          65                   1.681735   
                                                                                          60                  -0.328148   
                                                                                          50                  -1.069487   
                                                                                          40                  -2.640839   

                                                                                                                         \
                                                                                                                    std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 56                        NaN   
                                                                                          65                   4.220932   
                                                                                          60                   2.319143   
                                                                                          50                   3.451844   
                                                                                          40                   4.263029   

                                                                                                              annualized_return  \
                                                                                                                         median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                     
0.4                          5                          1.75           20                 56                           3.393979   
                                                                                          65                           1.425450   
                                                                                          60                           0.082520   
                                                                                          50                          -0.023027   
                                                                                          40                          -0.09621

In [146]:
df_highest_high_window_os

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,promotion_rank,promotion_set_id,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,out_sample,2023-10-01,2023-12-31,3.393979,4.790073,47.200647,0.498424,-0.071905,25 days,0.402174,2.484120,0.014817,36.0,0.4,Chandelier,1.75,20,65,5,0.703030,4.790073,True,1.0,fold1_2023-10-01,-0.081530,-1.470627,0.168078,-0.039915,-0.004522,-2.332985,0.065406,-0.006784,0.122786,0.849293,0.195297,-0.025211,0.719559,5.673582,0.124220,-0.010666,1.932473,4.207742,0.534999,-0.017236,0.246059
1,out_sample,2023-10-01,2023-12-31,3.393979,4.790073,47.200647,0.498424,-0.071905,25 days,0.402174,2.484120,0.014817,36.0,0.4,Chandelier,1.75,20,56,5,0.625946,4.790073,True,2.0,fold1_2023-10-01,-0.081530,-1.470627,0.168078,-0.039915,-0.004522,-2.332985,0.065406,-0.006784,0.122786,0.849293,0.195297,-0.025211,0.719559,5.673582,0.124220,-0.010666,1.932473,4.207742,0.534999,-0.017236,0.246059
2,out_sample,2024-01-01,2024-03-31,1.425450,3.378711,19.339829,0.400575,-0.073705,45 days,0.285714,1.781590,0.078189,22.0,0.4,Chandelier,1.75,20,65,5,1.937431,3.378711,True,1.0,fold2_2024-01-01,0.145278,1.120956,0.127998,-0.022401,1.174401,3.383170,0.326434,-0.079332,-0.014234,-8.831106,NaN,-0.003568,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.001438
3,out_sample,2024-01-01,2024-03-31,1.419399,3.312202,19.257735,0.366378,-0.073705,45 days,0.285714,1.746659,0.084108,22.0,0.4,Chandelier,1.75,20,50,5,1.867717,3.312202,True,2.0,fold2_2024-01-01,0.145284,1.121056,0.127995,-0.022401,1.175507,3.385061,0.326467,-0.079332,-0.010654,-3.509326,0.111660,-0.008481,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.084055
4,out_sample,2024-04-01,2024-06-30,-0.131261,-3.123579,-3.807588,0.099922,-0.034474,90 days,0.021978,-1.154181,0.251482,4.0,0.4,Chandelier,1.75,20,65,5,2.720630,-3.123579,True,1.0,fold3_2024-04-01,-0.140669,-2.722909,0.281513,-0.037091,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.750194
5,out_sample,2024-04-01,2024-06-30,-0.131261,-3.123579,-3.807588,0.099922,-0.034474,90 days,0.021978,-1.154181,0.251482,4.0,0.4,Chandelier,1.75,20,50,5,2.506930,-3.123579,True,2.0,fold3_2024-04-01,-0.140669,-2.722909,0.281513,-0.037091,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.750194
6,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,1.75,20,50,5,2.601236,0.373578,True,1.0,fold4_2024-07-01,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
7,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,1.75,20,40,5,2.535791,0.373578,True,2.0,fold4_2024-07-01,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
8,out_sample,2024-10-01,2024-12-31,0.335237,1.311734,4.552603,0.398203,-0.073636,38 days,0.228261,0.782412,0.436004,24.0,0.4,Chandelier,1.75,2

In [150]:
df_highest_high_window_is[df_highest_high_window_is.highest_high_window == 65]

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,promotion_rank,promotion_set_id,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
9,in_sample,2022-04-01,2023-09-30,0.068567,0.199168,0.801249,0.389036,-0.085575,304 days,0.078467,0.707643,0.479468,40.0,0.4,Chandelier,1.75,20,65,5,0.703030,NaN,False,NaN,None,0.051558,0.054576,0.172103,-0.047716,0.004564,-0.974029,0.149870,-0.032334,0.054954,0.100976,0.810916,-0.016114,-0.064026,-1.520319,0.338105,-0.113289,0.010765,-2.271355,0.105957,-0.004695,0.027410
19,in_sample,2022-07-01,2023-12-31,0.323277,1.442317,3.781325,0.407038,-0.085493,196 days,0.143898,2.120328,0.034427,74.0,0.4,Chandelier,1.75,20,65,5,1.937431,NaN,False,NaN,None,0.035445,-0.157088,0.170828,-0.085934,0.003898,-0.965155,0.137347,-0.033515,0.080826,0.409211,0.401369,-0.022937,0.045660,-0.023739,0.211204,-0.057161,0.159784,1.085476,0.415355,-0.025639,0.017594
29,in_sample,2022-10-01,2024-03-31,0.576878,2.121721,6.738905,0.427701,-0.085604,104 days,0.195255,2.897021,0.003918,93.0,0.4,Chandelier,1.75,20,65,5,2.720630,NaN,False,NaN,None,0.087605,0.463991,0.165577,-0.085828,0.143792,0.894448,0.233158,-0.079260,0.077956,0.374299,0.393758,-0.024901,0.044489,-0.038577,0.213215,-0.059028,0.157269,1.060589,0.486902,-0.026767,0.069252
39,in_sample,2023-01-01,2024-06-30,0.561725,2.054873,6.403244,0.427068,-0.087725,110 days,0.199269,2.809566,0.005139,93.0,0.4,Chandelier,1.75,20,65,5,2.362697,NaN,False,NaN,None,0.061313,0.164681,0.172664,-0.085839,0.144051,0.896210,0.233083,-0.079261,0.090122,0.524542,0.391751,-0.024900,0.050697,0.044343,0.209609,-0.052702,0.157317,1.060460,0.486883,-0.026455,0.067669
49,in_sample,2023-04-01,2024-09-30,0.428562,1.923954,4.306011,0.373664,-0.099527,204 days,0.149362,2.717404,0.006788,73.0,0.4,Chandelier,1.75,20,65,5,2.255701,NaN,False,NaN,None,-0.009791,-0.833094,0.165958,-0.085881,0.149827,0.980560,0.265191,-0.079270,0.034383,-0.378078,0.201568,-0.024802,0.087598,0.997611,0.117514,-0.012647,0.145668,0.952992,0.508803,-0.020527,0.065840
59,in_sample,2023-07-01,2024-12-31,0.494103,2.036883,3.639111,0.400393,-0.135776,254 days,0.170909,2.830963,0.004811,91.0,0.4,Chandelier,1.75,20,65,5,2.178805,NaN,False,NaN,None,0.071382,0.267315,0.208102,-0.043699,0.130634,0.844198,0.270199,-0.090214,0.059282,0.197295,0.209064,-0.024647,0.088923,1.020605,0.118811,-0.012671,0.126527,0.768268,0.481880,-0.043432,0.000983
69,in_sample,2023-10-01,2025-03-31,0.482021,1.884883,3.548737,0.421382,-0.135829,254 days,0.167883,2.620055,0.009036,96.0,0.4,Chandelier,1.75,20,65,5,2.220788,NaN,False,NaN,None,0.034832,-0.108220,0.209709,-0.075736,0.133272,0.863327,0.271708,-0.090209,0.046938,-0.033247,0.213530,-0.024471,0.093842,1.112651,0.121574,-0.012682,0.176522,1.103986,0.487912,-0.043522,0.053455


In [152]:
df_highest_high_window_is[df_highest_high_window_is.highest_high_window == 50]

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,promotion_rank,promotion_set_id,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
6,in_sample,2022-04-01,2023-09-30,0.073612,0.225778,0.816784,0.383738,-0.090124,284 days,0.096715,0.694587,0.487609,58.0,0.4,Chandelier,1.75,20,50,5,0.389649,NaN,False,NaN,None,0.067500,0.248304,0.197906,-0.047716,0.014423,-0.736916,0.138003,-0.032334,0.061726,0.188338,0.744148,-0.024377,-0.064632,-1.495579,0.268839,-0.114151,0.003361,-2.315211,0.117843,-0.013896,0.040655
16,in_sample,2022-07-01,2023-12-31,0.415075,1.687307,4.855156,0.395845,-0.085492,152 days,0.165756,2.389212,0.017222,92.0,0.4,Chandelier,1.75,20,50,5,1.867717,NaN,False,NaN,None,0.054113,0.087859,0.192884,-0.085934,0.013774,-0.734404,0.128644,-0.033515,0.087917,0.478810,0.404663,-0.024529,0.103413,0.611821,0.225940,-0.054712,0.151497,1.005389,0.407432,-0.025615,0.010386
26,in_sample,2022-10-01,2024-03-31,0.706139,2.379795,8.079896,0.425699,-0.087395,98 days,0.206204,3.195983,0.001474,100.0,0.4,Chandelier,1.75,20,50,5,2.506930,NaN,False,NaN,None,0.121694,0.751050,0.181549,-0.085828,0.143906,0.895341,0.233183,-0.079260,0.085762,0.451515,0.382779,-0.024805,0.097589,0.553923,0.254309,-0.056601,0.148877,0.979497,0.474463,-0.026766,0.064249
36,in_sample,2023-01-01,2024-06-30,0.686220,2.331449,7.823028,0.423445,-0.087718,110 days,0.204753,3.134778,0.001812,94.0,0.4,Chandelier,1.75,20,50,5,2.601236,NaN,False,NaN,None,0.094736,0.477936,0.187631,-0.085839,0.144163,0.897089,0.233107,-0.079261,0.090884,0.531586,0.376066,-0.023879,0.102565,0.616005,0.250816,-0.048339,0.157324,1.060523,0.486877,-0.026446,0.058613
46,in_sample,2023-04-01,2024-09-30,0.540876,2.219298,5.664395,0.384365,-0.095487,202 days,0.158470,3.054065,0.002367,74.0,0.4,Chandelier,1.75,20,50,5,2.558540,NaN,False,NaN,None,-0.009790,-0.833085,0.165957,-0.085881,0.149937,0.981416,0.265221,-0.079270,0.035086,-0.353058,0.195275,-0.023803,0.147712,1.473839,0.196102,-0.010664,0.171580,1.113378,0.490247,-0.022173,0.039089
56,in_sample,2023-07-01,2024-12-31,0.600949,2.273317,5.524642,0.413017,-0.108776,250 days,0.180000,3.099711,0.002036,93.0,0.4,Chandelier,1.75,20,50,5,2.583330,NaN,False,NaN,None,0.071384,0.267338,0.208104,-0.043699,0.130744,0.845119,0.270235,-0.090214,0.059969,0.208527,0.204846,-0.023693,0.137819,1.306642,0.203212,-0.020795,0.152369,0.942991,0.469126,-0.042370,0.032544
66,in_sample,2023-10-01,2025-03-31,0.588375,2.126003,4.431147,0.432522,-0.132782,250 days,0.177007,2.897453,0.003913,98.0,0.4,Chandelier,1.75,20,50,5,2.444467,NaN,False,NaN,None,0.034834,-0.108195,0.209711,-0.075736,0.133379,0.864221,0.271745,-0.090209,0.047587,-0.020303,0.208577,-0.023563,0.143121,1.366311,0.204364,-0.020834,0.203611,1.250529,0.475969,-0.042459,0.081304


## Highest High Window Walk Forward on a Micro Grid with Overlapping Timeframes

In [157]:
fixed = dict(
    # your frozen prod config passed into tf_fn
    fast_mavg=20, slow_mavg=200, mavg_stepsize=8, mavg_z_score_window=126,
    entry_rolling_donchian_window=56, exit_rolling_donchian_window=28,
    use_donchian_exit_gate=False,
    ma_crossover_signal_weight=0.9, donchian_signal_weight=0.1, weighted_signal_ewm_window=4,
    rolling_r2_window=100, lower_r_sqr_limit=0.45, upper_r_sqr_limit=0.9,
    r2_smooth_window=3, r2_confirm_days=0, r2_strong_threshold=0.75,
    log_std_window=14, coef_of_variation_window=20, vol_of_vol_z_score_window=126, vol_of_vol_p_min=0.10,
    use_activation=False, tanh_activation_constant_dict=None,
    moving_avg_type='exponential', long_only=True, price_or_returns_calc='price',
    initial_capital=15000, rolling_cov_window=20, volatility_window=30,
    stop_loss_strategy='Chandelier', rolling_atr_window=20,
    atr_multiplier=1.75, highest_high_window=56, cooldown_counter_threshold=5,
    annualized_target_volatility=0.40,  # fallback if sweep doesn't set it
    transaction_cost_est=0.001, passive_trade_rate=0.05,
    notional_threshold_pct=0.10, rolling_sharpe_window=50, cash_buffer_percentage=0.10,
    annual_trading_days=365, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-07-31',
    # use_specific_start_date=True, signal_start_date='2022-04-01'
)
sweep = {"highest_high_window": [50, 56, 60, 65]}

df_highest_high_window_overlap = run_wfa(
    start_date="2022-04-01", end_date="2025-07-31",
    ticker_list=['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD'],
    is_months=18, os_months=3, step_equals_os=False, warmup_days=300,
    fixed_params=fixed, sweep_params=sweep,
    tf_fn=tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol,
    perf_fn=calculate_risk_and_performance_metrics,
    include_ticker_metrics=True, promote_top_k=2
)


[WFA] init: IS=18m, OS=3m, STEP=1m, warmup=300d, tickers=5, grid_size=4, promote_top_k=2

[WFA] Fold 1: IS 2022-04-01 → 2023-09-30 | OS 2023-10-01 → 2023-12-31 | warmup=300d
[WFA] Fold 1: evaluating 4 config(s) in-sample…
[WFA] Fold 1 | IS cfg 1/4: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=1.75, rolling_atr_window=20, highest_high_window=50, cooldown_counter_threshold=5
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
[WFA] Fold 1 | IS cfg 1/4 METRICS: Sharpe=0.226, MDD=-9.012%, trades=58
[WFA] Fold 1 | IS cfg 2/4: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=1.75, rolling_atr_window=20, highest_high_window=56, cooldown_counter_threshold=5
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Si

In [159]:
df_highest_high_window_overlap['vol_tracking_error'] = (np.abs(df_highest_high_window_overlap['annualized_std_dev'] - df_highest_high_window_overlap['annualized_target_volatility']) / df_highest_high_window_overlap['annualized_target_volatility'])

df_highest_high_window_overlap = df_highest_high_window_overlap.replace([np.inf, -np.inf], np.nan)
in_sample_cond = (df_highest_high_window_overlap.sampling_category == 'in_sample')
df_highest_high_window_overlap_is = df_highest_high_window_overlap[in_sample_cond].reset_index(drop=True)
df_highest_high_window_overlap_os = df_highest_high_window_overlap[~in_sample_cond].reset_index(drop=True)

In [161]:
df_highest_high_window_overlap.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Highest_High_Window_Performance_Top_2_w_Overlap-2022-04-01-2025-10-01.pickle')
# df_atr_multiplier = pd.read_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Cooldown_Counter_Threshold_Performance-2022-04-01-2025-10-01.pickle')

In [163]:
df_highest_high_window_overlap.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier','rolling_atr_window','highest_high_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                                                               median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                           
0.4                          5                          1.75           20                 50                                 2.092252   
                                                                                          56                                 1.908069   
                                                                                          60                                 1.891077   
                                                                                          65                                 1.884883   

                                                                                                                         \
                                                                                                                   mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 50                   1.745412   
                                                                                          56                   1.538468   
                                                                                          60                   1.026091   
                                                                                          65                   0.980915   

                                                                                                                         \
                                                                                                                    std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 50                   0.773710   
                                                                                          56                   1.424506   
                                                                                          60                   2.469176   
                                                                                          65                   2.751784   

                                                                                                              annualized_return  \
                                                                                                                         median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                     
0.4                          5                          1.75           20                 50                           0.538594   
                                                                                          56                           0.465467   
                                                                                          60                           0.443849   
                                                                                          65                           0.431218   

                                                                                                                         \
                                                                                                                   mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 50                   0.468141   
                

In [165]:
df_highest_high_window_overlap_is.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier','rolling_atr_window','highest_high_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                                                               median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                           
0.4                          5                          1.75           20                 50                                 2.172577   
                                                                                          56                                 1.997991   
                                                                                          60                                 1.997991   
                                                                                          65                                 1.923954   

                                                                                                                         \
                                                                                                                   mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 50                   1.932664   
                                                                                          56                   1.755595   
                                                                                          60                   1.745611   
                                                                                          65                   1.692338   

                                                                                                                         \
                                                                                                                    std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 50                   0.588893   
                                                                                          56                   0.571022   
                                                                                          60                   0.599663   
                                                                                          65                   0.566996   

                                                                                                              annualized_return  \
                                                                                                                         median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                     
0.4                          5                          1.75           20                 50                           0.572753   
                                                                                          56                           0.498053   
                                                                                          60                           0.498053   
                                                                                          65                           0.482021   

                                                                                                                         \
                                                                                                                   mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 50                   0.513662   
                

In [167]:
df_highest_high_window_overlap_os.groupby(['annualized_target_volatility','cooldown_counter_threshold','atr_multiplier','rolling_atr_window','highest_high_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                                                                               median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                           
0.4                          5                          1.75           20                 50                                 0.728580   
                                                                                          60                                 0.551079   
                                                                                          56                                 0.373578   
                                                                                          65                                -0.812375   

                                                                                                                         \
                                                                                                                   mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 50                   0.559481   
                                                                                          60                   0.049599   
                                                                                          56                   0.949122   
                                                                                          65                  -0.370790   

                                                                                                                         \
                                                                                                                    std   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 50                   0.849519   
                                                                                          60                   3.567372   
                                                                                          56                   2.631038   
                                                                                          65                   4.451601   

                                                                                                              annualized_return  \
                                                                                                                         median   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window                     
0.4                          5                          1.75           20                 50                           0.191459   
                                                                                          60                           0.085207   
                                                                                          56                           0.085207   
                                                                                          65                           0.000000   

                                                                                                                         \
                                                                                                                   mean   
annualized_target_volatility cooldown_counter_threshold atr_multiplier rolling_atr_window highest_high_window             
0.4                          5                          1.75           20                 50                   0.179842   
                

In [179]:
df_highest_high_window_overlap_os.groupby(['highest_high_window','promotion_rank']).size()

highest_high_window  promotion_rank
50                   2.0               3
56                   1.0               4
                     2.0               5
60                   1.0               8
                     2.0               7
65                   1.0               7
                     2.0               4
dtype: int64

In [169]:
df_highest_high_window_overlap_os

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,promotion_rank,promotion_set_id,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,out_sample,2023-10-01,2023-12-31,3.393979,4.790073,47.200647,0.498424,-0.071905,25 days,0.402174,2.484120,0.014817,36.0,0.4,Chandelier,1.75,20,65,5,0.641867,4.790073,True,1.0,fold1_2023-10-01,-0.081530,-1.470627,0.168078,-0.039915,-0.004522,-2.332985,0.065406,-0.006784,0.122786,0.849293,0.195297,-0.025211,0.719559,5.673582,0.124220,-0.010666,1.932473,4.207742,0.534999,-0.017236,0.246059
1,out_sample,2023-10-01,2023-12-31,3.393979,4.790073,47.200647,0.498424,-0.071905,25 days,0.402174,2.484120,0.014817,36.0,0.4,Chandelier,1.75,20,56,5,0.382775,4.790073,True,2.0,fold1_2023-10-01,-0.081530,-1.470627,0.168078,-0.039915,-0.004522,-2.332985,0.065406,-0.006784,0.122786,0.849293,0.195297,-0.025211,0.719559,5.673582,0.124220,-0.010666,1.932473,4.207742,0.534999,-0.017236,0.246059
2,out_sample,2023-11-01,2024-01-31,1.959368,3.795822,25.279426,0.447519,-0.077508,36 days,0.413043,1.992059,0.049362,35.0,0.4,Chandelier,1.75,20,65,5,0.984991,3.795822,True,1.0,fold2_2023-11-01,-0.082638,-1.344686,0.158350,-0.039815,0.003148,-0.892325,0.106194,-0.012925,0.122448,0.846421,0.195112,-0.025207,0.391210,4.026124,0.115151,-0.010710,1.313887,3.603280,0.556446,-0.014683,0.118797
3,out_sample,2023-11-01,2024-01-31,1.959368,3.795822,25.279426,0.447519,-0.077508,36 days,0.413043,1.992059,0.049362,35.0,0.4,Chandelier,1.75,20,60,5,0.446792,3.795822,True,2.0,fold2_2023-11-01,-0.082638,-1.344686,0.158350,-0.039815,0.003148,-0.892325,0.106194,-0.012925,0.122448,0.846421,0.195112,-0.025207,0.391210,4.026124,0.115151,-0.010710,1.313887,3.603280,0.556446,-0.014683,0.118797
4,out_sample,2023-12-01,2024-02-29,1.940468,5.230879,54.044129,0.241654,-0.035905,54 days,0.373626,2.733099,0.007553,25.0,0.4,Chandelier,1.75,20,65,5,1.577093,5.230879,True,1.0,fold3_2023-12-01,0.263167,2.012759,0.136272,-0.031925,0.707976,4.093745,0.180713,-0.017134,0.214179,2.128682,0.156398,-0.022902,0.194465,2.226590,0.164917,-0.011162,0.000000,NaN,NaN,0.000000,0.395864
5,out_sample,2023-12-01,2024-02-29,1.940468,5.230879,54.044129,0.241654,-0.035905,54 days,0.373626,2.733099,0.007553,25.0,0.4,Chandelier,1.75,20,60,5,1.022889,5.230879,True,2.0,fold3_2023-12-01,0.263167,2.012759,0.136272,-0.031925,0.707976,4.093745,0.180713,-0.017134,0.214179,2.128682,0.156398,-0.022902,0.194465,2.226590,0.164917,-0.011162,0.000000,NaN,NaN,0.000000,0.395864
6,out_sample,2024-01-01,2024-03-31,1.425450,3.378711,19.339829,0.400575,-0.073705,45 days,0.285714,1.781590,0.078189,22.0,0.4,Chandelier,1.75,20,60,5,1.604369,3.378711,True,1.0,fold4_2024-01-01,0.145278,1.120956,0.127998,-0.022401,1.174401,3.383170,0.326434,-0.079332,-0.014234,-8.831106,NaN,-0.003568,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.001438
7,out_sample,2024-01-01,2024-03-31,1.425450,3.378711,19.339829,0.400575,-0.073705,45 days,0.285714,1.781590,0.078189,22.0,0.4,Chandelier,1.75,20,56,5,1.508753,3.378711,True,2.0,fold4_2024-01-01,0.145278,1.120956,0.127998,-0.022401,1.174401,3.383170,0.326434,-0.079332,-0.014234,-8.831106,NaN,-0.003568,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.001438
8,out_sample,2024-02-